# Migrating COM API calls to Python

### What are the COM APIs

The COM APIs are older APIs that were initially delivered with 3000 Xtra Terminals and then Eikon. They provided access to a broad set of content and capabilities in Eikon.

The COM API covers a large range of functionalities, as shown in [this article](https://developers.refinitiv.com/en/article-catalog/article/event-driven-financial-calculation-with-eikon-excel-visual-basic). Below we will show you how to migrate the following COM Component and API calls/functions: 



* AdfinXRtLib (rtx.dll)
    * [AdxRtSourceList]((#AdfinX)
    * [AdxRtList (including 'RtGet Real Time')](#RtList)
    * [AdxRtChain](#RtChain)
    * [AdxRtHistory (both inter- and intra-day) - Only available in Excel](#RtHistory)
    * [Streaming News](RtNews)


* dex2.dll 
    * [Rdata List](#RdataList)
    * [DEX2](#DEX2) (including Symbology Conversion, Time Series, Fundamental and Reference data) 

* RHistoryAPI.dll
    * [RHistory](#RHistory)
    
* RSearch.dll
    * [RSearch](#RSearch)

* RtContribute.dll 
    * [RtContribute](#RtContribute)


----The APIs below do not currently have replacements in Workspace---------------


* Adxfo.dll
    * AdfinX Analytics 2.0 Module


* Adxfoo.dll
    * AdfinX Analytics 3.0 Module




### Reasons you might need or want to migrate away from COM APIs:

### Need to operate in 64-bit environment
   
The COM APIs were designed to be used in 32-bit environments. Many teams are stil migrating to 64-bit environments where these will not work correctly and are not supported environments.

### You want access to new datasets and improved capabilties offered on Refinitiv Data Platform

The COM APIs are feature-complete and no new development work is being done on them. This means that new datasets    and capabilities will not be available to users of these APIs.

### You want to upgrade from Eikon to Workspace

The COM APIs will not be available in Workspace - so to take advantage of the best desktop experience you would need to migrate to our more modern web-based APIs - which are much better and work with modern scripting environments such as Python - one of the most popular language ecosystems used in finance. 


All the Python code below works in [Codebook](cpurl://apps.cp./Apps/code-book/) too! You can try it out as outlined in this [Tutorial Video](https://developers.refinitiv.com/en/video-catalog/from-developer-articles-to-codebook). We will use the Refinitiv Data Library (RD Library). To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [logging](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) into the Refinitiv Developer Community portal you will have free access to a number of learning materials like:
- [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
- [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
- [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)

including the ability to ask questions on our Q&A forums and much more.

## Prerequisites

### COM Prerequisites

Documentation on using the COM API in the Microsoft Office suite is available here: [COM APIs for Microsoft Office](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials). Users were also able to use the COM APIs outside of Microsoft Office suite for example in a standalone app: [COM APIs for use in custom applications](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-custom-applications). A list of the prerequisites in question can be found in the index of this article.

If you are new to [Python](https://www.python.org/), don't hesitate to install it on your machine and try it out yourself as outlined in this 3rd party [tutorial](https://pythonprogramming.net/introduction-learn-python-3-tutorials/). Otherwise, you can simply use [Codebook](cpurl://apps.cp./Apps/code-book/) as outlined in this [Tutorial Video](https://developers.refinitiv.com/en/video-catalog/from-developer-articles-to-codebook).

Python works with libraries that one can import to use functionalities that are not natively supported by the base coding package. Some popular distributuions of python include many of the popular packages that one could use for various tasks - [Anaconda](https://www.anaconda.com/) is the most popular such distribution. 

The RD Library allows for code portability across the desktop and enterprise platforms - with only small changes in authentication credentials. These credentials are stored in a config file - but if you are just using the desktop you need not concern yourself with this as a desktop session is the default credential setup.


In [28]:
import refinitiv.data as rd # pip install httpx==0.21.3 # !pip install refinitiv.data --upgrade
from refinitiv.data.discovery import Chain
from refinitiv.data.content import search
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import os
import time
import datetime  # `datetime` allows us to manipulate time as we would data-points.
from IPython.display import display, clear_output  # `IPython` here will allow us to plot grahs and the likes.

In [29]:
rd.open_session("desktop.workspace")

<refinitiv.data.session.Definition object at 0x7fa34230ac18 {name='workspace'}>

<a id='AdfinX'></a>
## AdfinX RealTime AdxRtSourceList
### What does 'AdfinX RealTime AdxRtSourceList' do?
This functionality was used to get the real-time field list for the selected Source Name, such as an instrument (e.g.: VOD.L) (using AdxRtSourceList Class of AdfinXRtLib, the AdfinX Real Time 6.0 Library, rtx.dll). It was best demonstrated in the [Tutorial 1 - Real-time Source List, Fields - AdxRtSourceList](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-1-real-time-source-list-fields-adx-rt-source-list)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/adxrtsourcelist.zip):

<img src="imgs/AdxRtSourceList.PNG">

### VBA

The VBA for AdxRtSourceList is very simple, and consists simply in creating an instance of an AdxRtSourceList object using the PLVbaApis function:

> Set myAdxRtSrcLst = CreateAdxRtSourceList()

### New Method
This functionality is replaced with a user interface called the [Data Item Browswer (DIB) that you can  find on workspace](https://emea1-apps.platform.refinitiv.com/web/Apps/DataItemBrowser/), and for which there is a [video tutorial](https://developers.refinitiv.com/en/video-catalog/data-item-browser). This will give you a list of all real-time and non-real-time fields available for a particular instrument. For real-time fields the RD library can also provide this programatically - see below:


#### Real-Time Fields Available With The COM API

There are a limited number of fields available on the COM API in 'AdxRtSourceList', you can lookup fields available in the Quote app for any instrument in question:

<img src="imgs/COM Real Time Fields1.PNG">

<img src="imgs/COM Real Time Fields2.PNG">

More are available on the DIB and via the code available below in the 'Real-Time Fields Available In Python' section.

Note that available fields for each instrument type difffers on the type. 

Note that the CF_ fields are also available to streaming Desktop sessions (not to platform sessions due to differences in data licenses)

#### Real-Time Fields Available In Python

You can get a collection of all the Real-Time fields available via:

In [30]:
RTCurrDf = rd.get_data(universe=['GBP='])
print(list(RTCurrDf.columns))

['Instrument', 'PROD_PERM', 'RDNDISPLAY', 'DSPLY_NAME', 'TIMACT', 'NETCHNG_1', 'HIGH_1', 'LOW_1', 'CURRENCY', 'ACTIV_DATE', 'OPEN_PRC', 'HST_CLOSE', 'BID', 'BID_1', 'BID_2', 'ASK', 'ASK_1', 'ASK_2', 'ACVOL_1', 'TRD_UNITS', 'PCTCHNG', 'OPEN_BID', 'OPEN_ASK', 'CLOSE_BID', 'CLOSE_ASK', 'NUM_MOVES', 'OFFCL_CODE', 'HSTCLSDATE', 'YRHIGH', 'YRLOW', 'BCKGRNDPAG', 'BID_NET_CH', 'BID_TICK_1', 'MID_PRICE', 'MID_NET_CH', 'MID_CLOSE', 'HIGHTP_1', 'LOWTP_1', 'BID_HIGH_1', 'BID_LOW_1', 'YRBIDHIGH', 'YRBIDLOW', 'HST_CLSBID', 'HSTCLBDDAT', 'NUM_BIDS', 'RECORDTYPE', 'ACT_TP_1', 'ACT_TP_2', 'ACT_TP_3', 'SEC_ACT_1', 'SEC_ACT_2', 'SEC_ACT_3', 'SC_ACT_TP1', 'SC_ACT_TP2', 'SC_ACT_TP3', 'OPEN_TIME', 'HIGH_TIME', 'LOW_TIME', 'YRHIGHDAT', 'YRLOWDAT', 'IRGPRC', 'TIMCOR', 'PRIMACT_1', 'PRIMACT_2', 'PRIMACT_3', 'BASE_CCY', 'BCAST_REF', 'CROSS_SC', 'DLG_CODE1', 'DLG_CODE2', 'DLG_CODE3', 'CTBTR_1', 'CTBTR_2', 'CTBTR_3', 'CTB_LOC1', 'CTB_LOC2', 'CTB_LOC3', 'CTB_PAGE1', 'CTB_PAGE2', 'CTB_PAGE3', 'VALUE_DT1', 'VALUE_DT

Note that available fields for each instrument type difffers on the type, e.g.:

In [31]:
ATMIVDf = rd.get_data(universe=['AAPLATMIV.U'])
print(list(ATMIVDf.columns))

['Instrument', 'PROD_PERM', 'RDNDISPLAY', 'DSPLY_NAME', 'RDN_EXCHID', 'TIMACT', 'CURRENCY', 'TRADE_DATE', 'TRDTIM_1', 'TRD_UNITS', 'RECORDTYPE', 'SALTIM', 'BEST_BID1', 'BEST_BID2', 'BEST_BID3', 'BEST_ASK1', 'BEST_ASK2', 'BEST_ASK3', 'BCAST_REF', 'BEST_BSIZ1', 'BEST_BSIZ2', 'BEST_BSIZ3', 'BEST_ASIZ1', 'BEST_ASIZ2', 'BEST_ASIZ3', 'LONGLINK1', 'GV1_TEXT', 'GV2_TEXT', 'GV4_TEXT', 'PREF_DISP', 'PREF_LINK', 'DSPLY_NMLL', 'DSO_ID', 'RDN_EXCHD2', 'PREV_DISP', 'UNDERLYING', 'TIMACT_MS', 'OPTIONS', '30D_A_IM_C', '30D_A_IM_P', '60D_A_IM_C', '60D_A_IM_P', '90D_A_IM_C', '90D_A_IM_P', 'CONTEXT_ID', 'CF_DATE', 'CF_EXCHNG', 'CF_SOURCE', 'CF_TIME', 'CF_NAME', 'DDS_DSO_ID', 'CF_CURR', 'SPS_SP_RIC']


<a id='RtList'></a>
## AdfinX RealTime AdxRtList

### What does AdxRtList do?

Returns real-time data for multiple (or single) instrument and fields. Data is returned initially as ONIMAGE - which is a snapshot of data for the requested fields, followed by a series of ONUPDATE messages - which are received whenever a data item changes or gets updated. You can also request ONTIME returns which would give an ONIMAGE snapshot at periodic intervals (say every hour for example). All the following functions rely on AdxRtList API calls - RtGet(), RData() and TR() for real-time data. Typically we would need to write callback handlers to deal with the returns from the API.

### RtGet Real-Time

When using the old COM API to get `RtGet Real Time` data, one may be greeted with an Excel sheet that looks like the below, as per [Tutorial 2 - Real-time Data Retrieval - AdxRtList](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-2-real-time-data-retrieval-adx-rt-list)'s [Excel workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/adxrtlist.zip):

<img src="imgs/1. RtGet Real Time Excel Front End Eg.PNG">

The functionalities shown here are easily recreated using the RD Library: 

1. Real Time FX rates snapshot - 'Real-Time ONIMAGE' 
2. 'Real Time Update', which updates real time, as soon as an update for the instrument and field is received 
3. A periodic ONTIME snapshot

#### Real-Time ONIMAGE

When collecting data ONIMAGE, we are collecting a current snapshot of the data we're after. This could not be simpler in RD in Python!

##### VBA

In VBA, you'd create an instance of an AdxRtList object using the PLVbaApis function CreateAdxRtList.

> Set myRtGet = CreateAdxRtList() ' The code will replicate RtGet(), one instrument, one field.'
> 
> Set myAdxRtList = CreateAdxRtList() ' The code will replicate RData(), multiple items & fields.'

Then create your function `cmdGetRealTimeONIMAGE_Click`:

>         Private Sub cmdGetRealTimeONIMAGE_Click()
>         Dim strRICs As String  ' Can have one or more items
>         Dim varFIDs As Variant ' Field can be numeric as well as a string, e.g. BID is field 22
>         
>         ActiveCell.Select
>         
>         If Not myRtGet Is Nothing Then Set myRtGet = Nothing
>         Set myRtGet = CreateAdxRtList()
>         
>         With myRtGet
>             .ErrorMode = DialogBox
>             .Source = [Source].Value
>             strRICs = [RIC].Value
>             varFIDs = [FID].Value
> 
>             .RegisterItems strRICs, varFIDs
>     '       'Different methods shown below.
>     '        strRICs = "EUR="
>     '        varFIDs = "BID"
>     '        .RegisterItems strRICs, varFIDs
>     '        .RegisterItems "EUR=,GBP=,JPY=", "BID,ASK"
> 
>             .StartUpdates RT_MODE_IMAGE ' 4
>             '.StartUpdates RT_MODE_ONUPDATE ' 3
>             '.StartUpdates RT_MODE_NOT_SET ' 5
>             '.StartUpdates RT_MODE_ONTIME ' 2
>             '.StartUpdates RT_MODE_ONTIME_IF_UPDATED ' 1
>         End With ' For the With myRtGet
>     End Sub

This would allow, in this example, for the 'Get Real Time ONIMAGE' buttons to work. For updates, the below could be used:

>     ' Returns the initial image for the instrument. NOTE - .StartUpdates RT_MODE_IMAGE
>     Private Sub myRtGet_OnImage(ByVal DataStatus As AdfinXRtLib.RT_DataStatus)
>         Dim arrRICs As Variant, arrFields As Variant
>         Dim lngRICFidVal As Single
>         Dim a As Integer
> 
>         If DataStatus = RT_DS_FULL Then
>             With myRtGet
>                 ' Array of the list of instruments - only one in this case.
>                 arrRICs = .ListItems(RT_IRV_ALL, RT_ICV_USERTAG)
>                 ' Array of the list of Fields for the ath item in the arrRics (base 0)
>                 a = 0
>                 arrFields = .ListFields(arrRICs(a, 0), RT_FRV_ALL, RT_FCV_VALUE)
>             End With
> 
>             ' And a specific value for a specific instrument, specific field.
>             'lngRICFidVal = myRtGet.Value("EUR=", "BID")
>             lngRICFidVal = myRtGet.Value([RIC].Value, [FID].Value)
>             [F7].Value = lngRICFidVal
>         End If
>     End Sub

##### Python

In python, things could not be easier as most of the equivalent code about is abstracted to the library and you just use a one-line function! You can go ahead and try it all out in [Codebook](cpurl://apps.cp./Apps/code-book/):

In [32]:
rd.get_data(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID', 'ASK'])

,Instrument,BID,ASK
0,GBP=,1.2002,1.2006
1,EUR=,1.0665,1.0666
2,JPY=,135.96,135.97


You can easily assign this info to an object too.

In [33]:
realTimeImage = rd.get_data(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID', 'ASK'])
realTimeImage

,Instrument,BID,ASK
0,GBP=,1.2002,1.2006
1,EUR=,1.0664,1.0667
2,JPY=,135.93,135.96


#### Real-Time ONUPDATE

##### VBA

<img src="imgs/1. RtGet Real Time Excel Front End Eg.PNG">

The 'Real Time ONUPDATE' buttons in the example pictured above was coded with VBA code for `cmdGetRealTimeONUPDATE_Click`:

>     Private Sub cmdGetRealTimeONUPDATE_Click()
>         Dim strRICs As Variant, varFIDs As Variant
>     
>          ActiveCell.Select
>         
>         Set myRtGet2 = CreateAdxRtList
>     
>         With myRtGet2
>             .ErrorMode = DialogBox
>             .Source = [Source].Value
>             strRICs = [RIC].Value
>             varFIDs = [FID].Value
>     
>             .RegisterItems strRICs, varFIDs
>             .StartUpdates RT_MODE_ONUPDATE
>         End With ' For the With myRtGet2
>     End Sub
>     
>     
>     ' Returns the data for updates - NOTE .StartUpdates RT_MODE_ONUPDATE.
>     Private Sub myRtGet2_OnUpdate(ByVal a_itemName As String, ByVal a_userTag As Variant, ByVal a_itemStatus As AdfinXRtLib.RT_ItemStatus)
>         Dim arrFields As Variant
>         Dim lngRICFidVal As Long
>     
>         If a_itemStatus = RT_ITEM_OK Then
>             arrFields = myRtGet2.ListFields(a_itemName, RT_FRV_ALL, RT_FCV_VALUE)
>     
>             ' And a specific value for a specific instrument, specific field.
>             'If a_itemName = "EUR=" Then lngRICFidVal = myRtGet2.Value("EUR=", "BID"): [F12].Value = arrFields(0, 1)
>             If a_itemName = [RIC].Value Then lngRICFidVal = myRtGet2.Value([RIC].Value, [FID].Value): [F12].Value = arrFields(0, 1)
>         End If
>     End Sub

Then, on VBA, you'd have to have a buttons to stop the stream with `cmdSwitchRealTimeOFF_Click`, which, in python, is `stream.close()`:

##### Python

For us to start using pricing streams with events, we need to define a callback to receive data events:

In [34]:
def display_data(data, instrument, stream):
    clear_output(wait=True)
    current_time = datetime.datetime.now().time()
    print(current_time, "- Data received for", instrument)
    display(data)

Open the stream and register the callback

In [35]:
stream = rd.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID', 'ASK'],
    on_data=display_data
)

11:38:32.995630 - Data received for EUR=


Close the stream

In [36]:
stream.close()

,BID,ASK
EUR=,1.0663,1.0667


<OpenState.Closed: 'Closed'>

More related RD capabilities can be found [here](https://github.com/Refinitiv-API-Samples/Example.DataLibrary.Python/blob/main/Examples/1-Access/EX-1.01.03-PricingStream.ipynb).

#### Real-Time ONTIME

With Real-Time ONTIME, you can get snapshot updates at a specific frequency:

<img src="imgs/1. RtGet Real Time Excel Front End Eg.PNG">

We can use a Python loop with `sleep` to recreate that simply:

The cell below gets an update for instruments 'GBP=', 'EUR=' and 'JPY=' and fields 'BID' and 'ASK' every 5 seconds:

In [37]:
# This cell's code is usually commented out so that the kernel doesn't get stuck in the while loop.
now = time.perf_counter()
while time.perf_counter() < now + 30:
    time.sleep(5)
    clear_output(wait=True)
    df = stream.get_snapshot(
        universe=['GBP=', 'EUR=', 'JPY='],
        fields=['BID', 'ASK'])
    display(df)

,Instrument,BID,ASK
0,GBP=,1.2003,1.2006
1,EUR=,1.0663,1.0667
2,JPY=,135.96,135.97


### NEW FUNCTIONALITY: Record ticks

With the RD library we now have the ability to record a pricing stream. Here's how:

Create and open a Pricing stream

In [38]:
stream = rd.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY='],
    fields=['BID']
)

Start recording

In [39]:
stream.recorder.record(frequency='tick')

... Wait for a little while (5 seconds) ...

In [40]:
time.sleep(5)

Stop recording and display the recorded history

In [41]:
stream.recorder.stop()
tick_history = stream.recorder.get_history()
display(tick_history)

,GBP=,JPY=,EUR=
,BID,BID,BID
Timestamp,,,
2023-03-07 11:40:43.153856,<NA>,135.97,<NA>
2023-03-07 11:40:43.312900,<NA>,<NA>,1.0661
2023-03-07 11:40:43.473794,<NA>,135.96,<NA>
2023-03-07 11:40:43.494152,1.1995,<NA>,<NA>
2023-03-07 11:40:43.617260,<NA>,<NA>,1.0661
2023-03-07 11:40:43.759178,<NA>,135.97,<NA>
2023-03-07 11:40:43.813662,1.1995,<NA>,<NA>
2023-03-07 11:40:44.265636,<NA>,135.95,<NA>


Resample the tick history to 5 seconds bars

In [42]:
tick_history.ohlc("5s")

GBP=                            JPY=                  \
                        BID                             BID                   
                       open    high     low   close    open    high     low   
Timestamp                                                                     
2023-03-07 11:40:40  1.1995  1.1995  1.1995  1.1995  135.97  135.97  135.95   
2023-03-07 11:40:45  1.1995  1.1995  1.1994  1.1994  135.95  135.97  135.95   
2023-03-07 11:40:50  1.1994  1.1994  1.1994  1.1994  135.95  135.95  135.95   

                               EUR=                          
                                BID                          
                      close    open    high     low   close  
Timestamp                                                    
2023-03-07 11:40:40  135.95  1.0661  1.0661  1.0661  1.0661  
2023-03-07 11:40:45  135.95  1.0661  1.0661  1.0661  1.0661  
2023-03-07 11:40:50  135.95  1.0661  1.0661  1.0661  1.0661

Close the stream

In [43]:
stream.close()

<OpenState.Closed: 'Closed'>

<a id='RtChain'></a>
## AdfinX RealTime - AdxRtChain

### What does AdxRtChain do?

As per [Tutorial 3 - Real-time Chain Retrieval - AdxRtChain](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-3-real-time-chain-retrieval-adx-rt-chain)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/adxrtchain.zip), Adfin X RealTime Chain (AdxRtChain) returns a list of the constituent instrument codes for any chain such as 0#.FTSE (the FTSE 100 instruments). Data is returned as OnUpdate event, the only other event is OnStatusChange:

<img src="imgs/AdxRtChain.PNG">

### VBA

In VBA, we went through with the creation of `cmdGetChain_Click`:

>     Private Sub cmdGetChain_Click()
>         ActiveCell.Select
>         
>         If myAdxRtChain Is Nothing Then Set myAdxRtChain = CreateAdxRtChain()
>         
>         With myAdxRtChain
>             .Source = "IDN"
>             .ItemName = Range("G6").Value
>             .RequestChain
>         End With
>     End Sub

then `myAdxRtChain_OnUpdate`:

>     Private Sub myAdxRtChain_OnUpdate(ByVal DataStatus As AdfinXRtLib.RT_DataStatus)
>         Dim i As Integer
>         
>         If DataStatus = RT_DS_FULL Then
>             For i = 1 To UBound(myAdxRtChain.Data)
>                 Range("G8").Offset(i - 1, 0).Value = myAdxRtChain.Data(i)
>             Next i
>         End If
>     End Sub

then we ought to make sure we can close the connection with `cmdClearChain_Click`, which is done simply in Python with `rd.close_session()`.

We can replicate this easily in Python with the Pricing snapshots and Fundamental & Reference data function `get_data()` - moreover we can decode the chain and request fields in one operation:

### Python

In [44]:
FTSEConstituentDf1 = rd.get_data(
    universe=['0#.FTSE'],
    fields=['TR.TURNOVER.timestamp', 'TR.TURNOVER', 'TR.EVToSales'])
FTSEConstituentDf1

,Instrument,Timestamp,Turnover,Enterprise Value To Sales (Daily Time Series Ratio)
0,STAN.L,2023-03-06T00:00:00Z,5254690524.0795,3.641311
1,CRDA.L,2023-03-06T00:00:00Z,1274351304.23038,4.528984
2,ANTO.L,2023-03-06T00:00:00Z,2642480584.4064,3.978246
3,BNZL.L,2023-03-06T00:00:00Z,1357517624.49449,0.956368
4,SGE.L,2023-03-06T00:00:00Z,1904311639.98092,4.387063
...,...,...,...,...
95,EDV.L,2023-03-06T00:00:00Z,298534187.4035,2.206513
96,OCDO.L,2023-03-06T00:00:00Z,1123284787.22775,2.010882
97,LSEG.L,2023-03-06T00:00:00Z,2993876995.24628,6.336949
98,TSCO.L,2023-03-06T00:00:00Z,3041956489.34064,0.468989


#### Not all chains resolve directly - for example a commodity chain - in such cases we can use the Chain Object to decode as follows:

In [45]:
LCOConstituentDf = rd.get_data(
    universe=Chain('0#LCO:'),
    fields=["CF_NAME", "CF_CLOSE", "OPINT_1"])

LCOConstituentDf

,Instrument,CF_NAME,CF_CLOSE,OPINT_1
0,LCOTOT,BRENT CRUDE VOLS,<NA>,2459512
1,LCOK3,BRENT CRUDE MAY3,86.18,467381
2,LCOM3,BRENT CRUDE JUN3,85.62,448239
3,LCON3,BRENT CRUDE JUL3,85.16,182689
4,LCOQ3,BRENT CRUDE AUG3,84.69,111730
...,...,...,...,...
79,LCOX9,BRENT CRUDE NOV9,65.08,<NA>
80,LCOZ9,BRENT CRUDE DEC9,65.06,<NA>
81,LCOF0,BRENT CRUDE JAN0,65.04,<NA>
82,LCOG0,BRENT CRUDE FEB0,65.02,<NA>


<a id='RtHistory'></a>
## AdfinX RealTime - AdxRtHistory - Interday Time Series History

### What does AdxRtHistory do?

Adfin RealTime History (AdxRtHistory) is used to retrieve interday (not intraday) time series (historic) data for an instrument or instruments. This was best exemplified in [Tutorial 5 - Time Series History - AdxRtHistory](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-5-time-series-history-adx-rt-history)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/adxrthistory.zip):


<img src="imgs/AdxRtHistory Interday Time Series History Eg.PNG">

### VBA

In VBA, we used `AdfinXRtLib`:

>     ' Note the use of CreateReutersObject - function in the PLVbaApis module.
>         If myAdxRtHist Is Nothing Then Set myAdxRtHist = CreateReutersObject("AdfinXRtLib.AdxRtHistory")
>         
>         On Error GoTo errHndlr
>         With myAdxRtHist
>             .FlushData
>             .ErrorMode = EXCEPTION ' EXCEPTION, DialogBox, NO_EXCEPTION
>             .Source = "IDN"
>             .ItemName = [C7].Value
>             .Mode = [H8].Value
>             .RequestHistory ("DATE,CLOSE,VOLUME") 'NOTE USE OF OLD FIELD NAMES, NOT ("TRDPRC_1.TIMESTAMP,TRDPRC_1.CLOSE,TRDPRC_1.VOLUME")
>             
>             'arrFlds = Array("DATE","CLOSE","VOLUME")
>             '.RequestHistory ()arrFlds
>             '.RequestHistory ("*") ' "*" requests all fields.
>         End With

before the `Private Sub` '`myAdxRtHist_OnUpdate(ByVal DataStatus As AdfinXRtLib.RT_DataStatus)`'.

Things are simpler in Python:

### Python

As aforementioned, AdxRtHistory is used to retrieve time series (historic) data for an instrument or instruments except for intraday data. This is exactly what the instruments `get_history` is for!

In [40]:
FTSEConstituents = list(FTSEConstituentDf1['Instrument'])
print(FTSEConstituents)

['STAN.L', 'CRDA.L', 'ANTO.L', 'BNZL.L', 'SGE.L', 'SVT.L', 'BLND.L', 'ICAG.L', 'REL.L', 'SMIN.L', 'AZN.L', 'HSBA.L', 'CTEC.L', 'WPP.L', 'FRES.L', 'AAF.L', 'SGRO.L', 'SJP.L', 'TW.L', 'AHT.L', 'HLMA.L', 'III.L', 'CNA.L', 'MNG.L', 'BKGH.L', 'SMDS.L', 'NG.L', 'RKT.L', 'SKG.L', 'WEIR.L', 'MRON.L', 'HSX.L', 'CPG.L', 'AUTOA.L', 'AV.L', 'ENT.L', 'DGE.L', 'INF.L', 'UU.L', 'PSHP.L', 'HLN.L', 'WTB.L', 'PRU.L', 'IMB.L', 'EXPN.L', 'BRBY.L', 'RS1R.L', 'ABDN.L', 'GSK.L', 'LAND.L', 'BEZG.L', 'BP.L', 'JD.L', 'ABF.L', 'AAL.L', 'ADML.L', 'RTO.L', 'RMV.L', 'SBRY.L', 'PHNX.L', 'FLTRF.L', 'IHG.L', 'BT.L', 'MNDI.L', 'BATS.L', 'PSON.L', 'CRH.L', 'SPX.L', 'PSN.L', 'RIO.L', 'JMAT.L', 'CCH.L', 'RR.L', 'SN.L', 'BMEB.L', 'SSE.L', 'SMT.L', 'FRAS.L', 'HRGV.L', 'KGF.L', 'LLOY.L', 'NWG.L', 'SDR.L', 'NXT.L', 'ITRK.L', 'BDEV.L', 'SHEL.L', 'GLEN.L', 'VOD.L', 'BARC.L', 'FCIT.L', 'UTG.L', 'BAES.L', 'DCC.L', 'ULVR.L', 'EDV.L', 'OCDO.L', 'LSEG.L', 'TSCO.L', 'LGEN.L']


In [41]:
FTSEConstituents[0:6]

['STAN.L', 'CRDA.L', 'ANTO.L', 'BNZL.L', 'SGE.L', 'SVT.L']

In [42]:
TimeSeriesDf = rd.get_history(
    universe=FTSEConstituents[1:6],
    fields=['TR.PriceClose', 'TR.Volume'],  # 'TR' fields are usually historic ones.
    interval="1D",
    start="2022-01-25",
    end="2022-02-01")

TimeSeriesDf

CRDA.L              ANTO.L               BNZL.L           \
           Price Close  Volume Price Close   Volume Price Close   Volume   
Date                                                                       
2022-01-25        7666  371438      1379.0  1429044        2721   678948   
2022-01-26        7830  755599      1411.5   889172        2754   628530   
2022-01-27        7880  754142      1391.5  1460311        2776   671478   
2022-01-28        7774  510683      1337.5  2839242        2786  1203491   
2022-01-31        7972  442252      1332.5  1333271        2767   545449   
2022-02-01        8008  690445      1361.5  1605104        2775   433783   

                 SGE.L                SVT.L           
           Price Close   Volume Price Close   Volume  
Date                                                  
2022-01-25       762.4  3467017        2858   642275  
2022-01-26       713.0  4323758        2871   318280  
2022-01-27       701.0  4752740        2914   522647  
2022-01-28       711.2  3683775        2893  1017099  
2022-01-31       720.8  2966860        2873   751851  
2022-02-01       718.6  4462156        2903   439666

## AdxRtHistory - Intraday Time Series History

### VBA

Adfin RealTime History (AdxRtHistory) Intraday is similar

Which had few VBA lines needed:

>     Private Sub cmdGetInterday_Click()
>         ActiveCell.Select
>         
>         MsgBox "AdxRtHistory cannot retrieve INTRA day data, use the RHistoryAPI instead"
>     End Sub


### Python

Intraday data is just as easy to get:

In [51]:
IntradayTimeSeriesDf = rd.get_history(
    universe=FTSEConstituents,
    fields=['TRDPRC_1'],
    interval="1min",  # The consolidation interval. Supported intervals are: tick, tas, taq, minute, 1min, 5min, 10min, 30min, 60min, hourly, 1h, daily, 1d, 1D, 7D, 7d, weekly, 1W, monthly, 1M, quarterly, 3M, 6M, yearly, 1Y.
    start="2022-06-01T13:00:00",
    end="2022-06-01T15:30:00")

IntradayTimeSeriesDf

TRDPRC_1,STAN.L,CRDA.L,ANTO.L,BNZL.L,SGE.L,SVT.L,BLND.L,ICAG.L,REL.L,SMIN.L,AZN.L,HSBA.L,CTEC.L,WPP.L,FRES.L,AAF.L,SGRO.L,SJP.L,TW.L,AHT.L,HLMA.L,III.L,CNA.L,MNG.L,BKGH.L,SMDS.L,NG.L,RKT.L,SKG.L,WEIR.L,MRON.L,HSX.L,CPG.L,AUTOA.L,AV.L,ENT.L,DGE.L,INF.L,UU.L,PSHP.L,HLN.L,WTB.L,PRU.L,IMB.L,EXPN.L,BRBY.L,RS1R.L,ABDN.L,GSK.L,LAND.L,BEZG.L,BP.L,JD.L,ABF.L,AAL.L,ADML.L,RTO.L,RMV.L,SBRY.L,PHNX.L,FLTRF.L,IHG.L,BT.L,MNDI.L,BATS.L,PSON.L,CRH.L,SPX.L,PSN.L,RIO.L,JMAT.L,CCH.L,RR.L,SN.L,BMEB.L,SSE.L,SMT.L,FRAS.L,HRGV.L,KGF.L,LLOY.L,NWG.L,SDR.L,NXT.L,ITRK.L,BDEV.L,SHEL.L,GLEN.L,VOD.L,BARC.L,FCIT.L,UTG.L,BAES.L,DCC.L,ULVR.L,EDV.L,OCDO.L,LSEG.L,TSCO.L,LGEN.L
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-06-01 13:00:00,636.8,<NA>,<NA>,2785.0,652.2,2855.0,526.506,129.2256,<NA>,1570.0,10492.0,535.4,<NA>,937.2,<NA>,<NA>,1096.0,1256.5,130.95,4115.0,2198.0,<NA>,80.0871,<NA>,4253.0,306.7,1127.4703,<NA>,3221.0,1620.5,134.6,924.8,1811.5,587.6,429.8,<NA>,3662.36,545.0,<NA>,<NA>,<NA>,2721.0,1022.4673,1811.0,2602.0,1734.5,969.5,<NA>,1745.185988,<NA>,488.0,433.308,123.9,1730.0,3869.0,<NA>,500.4,588.2,<NA>,<NA>,9450.0,<NA>,189.9,1546.5,3568.5,759.4,<NA>,10520.0,<NA>,5781.0,2139.0,1729.5,89.1448,<NA>,376.17986,1786.0,812.8979,699.5,846.6,264.5,45.625,<NA>,<NA>,6504.0,4612.0,<NA>,2370.0,515.386207,127.26,171.36,<NA>,1135.0,777.16,5606.0,3759.7,1794.0,<NA>,7252.004,260.87,258.3
2022-06-01 13:01:00,636.8,6884.0,<NA>,2784.0,<NA>,2854.0,526.6,128.74,2242.0,<NA>,10492.0,535.5,217.8,937.4,<NA>,<NA>,<NA>,<NA>,130.9,4114.0,<NA>,1251.5725,80.14,<NA>,4256.0,306.768,1127.35,<NA>,3223.0,1619.5,134.55,<NA>,1811.5,587.8,430.1,<NA>,3663.5,545.2,1049.61,<NA>,<NA>,<NA>,1022.0,1810.5,2605.3468,1733.5,<NA>,<NA>,1746.398063,770.4,<NA>,433.5681,123.9,<NA>,3868.0,<NA>,501.0,589.2,230.0,<NA>,9442.0,<NA>,189.716,1547.0,3566.0,<NA>,3272.5,10525.0,<NA>,5781.0,2140.0,<NA>,89.18,<NA>,376.3712,1786.5,<NA>,700.5,846.6,<NA>,45.6034,231.910913,<NA>,<NA>,4613.0,<NA>,2371.0,515.092205,<NA>,171.42,<NA>,1136.0,777.5818,5608.0,3760.4058,1793.0,920.0,7253.7679,260.8942,258.4
2022-06-01 13:02:00,636.6,6882.0,1507.0,<NA>,<NA>,<NA>,526.4,128.64,2242.0,1570.78,10494.0,535.2998,<NA>,937.0804,<NA>,<NA>,<NA>,1258.0,131.0,<NA>,2202.0,<NA>,80.12,218.05,4258.0,<NA>,1128.0,<NA>,<NA>,1620.5,<NA>,<NA>,1811.5,<NA>,429.9026,<NA>,3664.0,544.8,<NA>,<NA>,<NA>,<NA>,1022.3735,1811.0,2611.0,<NA>,<NA>,194.55,1746.19605,<NA>,<NA>,433.55,123.7805,1730.0,3869.0,<NA>,500.7978,589.8,230.1,637.4,<NA>,4933.0,189.819,<NA>,<NA>,<NA>,<NA>,<NA>,2192.0,5779.0,2140.6082,1729.5,89.2358,<NA>,376.753496,1787.0,812.899,<NA>,846.0,<NA>,45.5725,<NA>,498.1,6504.0,4615.0,505.8,2371.0,514.994204,127.28,171.28,<NA>,<NA>,777.6,<NA>,3760.0,1793.0,920.2,7252.0,260.8139,258.4
2022-06-01 13:03:00,<NA>,6890.0,<NA>,2787.0,652.6,2854.0,526.8,128.68,2243.0,1570.5,10494.0,535.3982,218.0,937.8,<NA>,<NA>,<NA>,<NA>,131.05,4121.0,2209.0,1252.2297,80.12,<NA>,<NA>,<NA>,<NA>,6252.0,3226.0,1621.0,134.55,<NA>,1811.5,588.8,<NA>,1469.0,3665.5,544.8,1049.6095,<NA>,<NA>,2721.0,1022.0,1811.2495,2618.0,<NA>,970.0,<NA>,1746.802088,<NA>,<NA>,433.7,<NA>,1729.5,3872.0,<NA>,501.4,589.8,230.11,637.4,9440.0,4934.0,<NA>,<NA>,3564.6904,759.6,<NA>,10535.0,<NA>,5782.5,2139.8041,1730.0,89.22,1295.0,377.040219,1787.2995,813.698,<NA>,846.0,<NA>,45.5894,231.609469,<NA>,<NA>,4618.0,505.8,2371.0,515.288206,127.283,171.38,<NA>,<NA>,778.16,5610.0,3759.5,1794.0,922.6,7252.0,260.9699,258.4
2022-06-01 13:04:00,636.4,6892.0,1509.0,2785.0,653.0,2853.0,<NA>,128.74,2244.0,1572.0,10495.7,535.3,<NA>,<NA>,770.2,<NA>,<NA>,1259.5,<NA>,4124.0,2210.0,1252.0,80.1,<NA>,4258.0,306.5,1128.3498,6248.0,3228.0,1622.0,134.7,<NA>,1811.5,589.2,429.5,1470.5,3667.5,545.0,1049.0,<NA>,<NA>,2722.0,<NA>,1810.5,2623.0,1734.0,971.5,194.675,1746.600075,771.0,<NA>,433.7,123.938,<NA>,3874.0,<NA>,501.491,590.4,<NA>,<NA>,9434.0,4931.0,189.8,<NA>,3563.0,<NA>,3272.5,<NA>,2193.417,5786.0,<NA>,1730.7056,89.2299,1296.0,377.51809,1786.8,813.2,700.5,847.6,<NA

As you can see, we get the data sought after. If you are after several fields for any one instrument (e.g.: an FX Pair), it couldn't be any simpler.

## DEX2.dll

The DEX2.dll COM API component provides access to a broad range of fundamental and reference data (including all the TR.xxx fields). The RData Excel function provided both Fundamental and Reference as well as streaming realtime prices and news using this component under the hood along with RTX.dll.

<a id='RdataList'></a>
### Rdata List Realtime

When using the old COM API to get `Rdata List` data, one may be greeted with an Excel sheet that looks like this:

<img src="imgs/2. RData Excel Front End Eg.PNG">

#### VBA

In VBA, this was done with a function akin to `.StartUpdates RT_MODE_ONUPDATE` & `myRTList = CreateAdxRtList()`, e.g.:


>     With myRTList
>         .ErrorMode = EXCEPTION
>         ' N.B.! Source name may need to be changed if not named as below!
>         .Source = "IDN" '_SELECTFEED"
>         ' Register the items and fields
>         .RegisterItems ItemArray, FieldArray
> 
>         ' Set the user tag on each item. This helps indexing the results
>         ' table for displaying the data in the callback
>         For m = LBound(ItemArray) To UBound(ItemArray)
>             .UserTag(ItemArray(m), "*") = m
>             For n = LBound(FieldArray) To UBound(FieldArray)
>                 .UserTag(ItemArray(m), FieldArray(n)) = n
>             Next n
>         Next m
> 
>         .Mode = "TIMEOUT:5"
>         ' If timed basis desired, then FRQ setting and RT_MODE_ONTIME or RT_MODE_ONTIME_IF_UPDATED required,
>         ' which will trigger the OnUpdate event, shown below.
>         '.Mode = "FRQ:2S"
>         ' And, finally, request the data!
>         Select Case Range("dcUpdateType").Value
>             Case "RT_MODE_IMAGE"
>                 .StartUpdates RT_MODE_IMAGE
>             Case "RT_MODE_ONUPDATE"
>                 .StartUpdates RT_MODE_ONUPDATE
>         End Select
> 
>         '.StartUpdates RT_MODE_ONUPDATE
>         '.StartUpdates RT_MODE_IMAGE
>         'Other modes shown below; different events will be fired.
>         '.StartUpdates RT_MODE_ONTIME, RT_MODE_ONTIME_IF_UPDATED, RT_MODE_ONTIME,
>         ' RT_MODE_ONUPDATE, RT_MODE_IMAGE , RT_MODE_NOT_SET
>     End With

To stop this update, you would have to create some VBA code to (e.g.: `Sub cmdStop_Click()`), but that is simpler in Python with `stream.close()`:

However - many developers also used the RData worksheet function object directly in VBA. 

#### Python

Here we have a data-frame of instruments and fields updating live every `x` seconds, let's say (for the sake of the use-case example) every 3 seconds. This is simple to recreate in Python:

In [46]:
#define stream
stream = rd.open_pricing_stream(
    universe=['GBP=', 'EUR=', 'JPY=', '.GDAXI', '.FTSE', '.NDX', 'TRI.TO', 'EURGBP=R'],
    fields=['CF_TIME', 'CF_LAST', 'BID', 'ASK', 'TRDTIM_1'])

In [47]:
#open stream
stream.open()

<OpenState.Opened: 'Opened'>

In [48]:
#add temporal update functionality using stream.get_snapshot 
now = time.perf_counter()
while time.perf_counter() < now + 30:
    time.sleep(3)
    clear_output(wait=True)
    df = stream.get_snapshot(
        universe=['GBP=', 'EUR=', 'JPY=', '.GDAXI', '.FTSE', '.NDX', 'TRI.TO', 'EURGBP=R'], 
        fields=['CF_TIME', 'CF_LAST', 'BID', 'ASK', 'TRDTIM_1'])
    display(df)

,Instrument,CF_TIME,CF_LAST,BID,ASK,TRDTIM_1
0,GBP=,11:44:36,1.1983,1.1983,1.1987,<NA>
1,EUR=,11:44:36,1.0657,1.0657,1.0661,<NA>
2,JPY=,11:44:37,136.02,136.02,136.03,<NA>
3,.GDAXI,11:44:00,15677.13,<NA>,<NA>,11:44:00
4,.FTSE,11:44:00,7948.93,<NA>,<NA>,11:44:00
5,.NDX,22:15:59,12302.484,<NA>,<NA>,<NA>
6,TRI.TO,21:00:00,165.81,162.61,167,<NA>
7,EURGBP=R,11:44:37,0.889,0.889,0.8897,<NA>


Close the stream

In [49]:
stream.close()

<OpenState.Closed: 'Closed'>

#### Create a Streaming Price and register event callbacks using RDP

You can build upon the example above, using the [RDP](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis) Example notebook present in [Codebook](cpurl://apps.cp./Apps/code-book/) that demonstrates how to use a StreamingPrice with events to update a Pandas DataFrame with real-time streaming data. Using a StreamingPrices object that way allows your application to have at its own in memory representation (a Pandas DataFrame in this example) that is kept updated with the latest streaming values received from Eikon or Refinitiv Workspace. Here we're putting ourselves in the shoes of a Foreign eXchange (FX) trader looking at Emerging Market (EM) currency exchange rates; e.g: the Nigerian Nairas (NGN) and Indonesian Rupiah (IDR).

You can find the code for this on GitHub [here](https://gist.github.com/johnukfr/25935ae4a0928c400e706dee7eb6a79a).

<a id='RdataFunction'></a>
### RData Function

#### What does RData do?
RData is a flexible excel worksheet function allowing access to realtime and fundamental & reference data content. It can also be used programatcally in VBA and the results then dumped to an excel range for example. 

#### VBA

For VBA related to Fundamental data, please see the 'DEX2 Fundamental and Reference' section below.

#### Python

We have separated getting current fundamental snapshots - using a rd.get_data function and getting historical fundamental timeseries using either the rd.get_data function or the rd.get_history() function. 

##### Snapshot requests

For snapshot current fundamental requests - things are pretty straight forward - select your universe of instruments and then the list of fields you want. A full list of fields is available using the [**Data Item Browser App**](https://developers.refinitiv.com/en/video-catalog/data-item-browser) (type DIB into Eikon or Workspace search bar).

In [50]:
df1 = rd.get_data(
    universe=['BARC.L', 'TRI.N', 'TSLA.O'],
    fields=['TR.RevenueMean.date', 'TR.RevenueMean', 'TR.TRBCEconomicSector',
            'TR.TRBCEconSectorCode', 'TR.TRBCBusinessSector',
            'TR.TRBCBusinessSectorCode', 'TR.TRBCIndustryGroup',
            'TR.TRBCIndustryGroupCode', 'TR.TRBCIndustry', 'TR.TRBCIndustryCode'])
df1

,Instrument,Date,Revenue - Mean,TRBC Economic Sector Name,TRBC Economic Sector Code,TRBC Business Sector Name,TRBC Business Sector Code,TRBC Industry Group Name,TRBC Industry Group Code,TRBC Industry Name,TRBC Industry Code
0,BARC.L,2023-03-06,26185926370,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
1,TRI.N,2023-02-22,6937020650,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070
2,TSLA.O,2023-03-05,103133758370,Consumer Cyclicals,53,Automobiles & Auto Parts,5310,Automobiles & Auto Parts,531010,Auto & Truck Manufacturers,53101010


If we want to add some fundamental history to this request - we can add a parameters section to the get_data request - as below which will give us the last 4 fiscal years ('FRQ': 'FY') of history for each RIC. Note for static reference fields such sector codes - these will not be published as a timeseries history - however, we can forward fill as shown below.

In [51]:
df1 = rd.get_data(
    universe=['BARC.L', 'TRI.N', 'TSLA.O', Peers('HD'), Customers],
    fields=[
        'TR.RevenueMean.date', 'TR.RevenueMean',
        'TR.TRBCEconomicSector', 'TR.TRBCEconSectorCode', 'TR.TRBCBusinessSector',
        'TR.TRBCBusinessSectorCode', 'TR.TRBCIndustryGroup', 'TR.TRBCIndustryGroupCode',
        'TR.TRBCIndustry', 'TR.TRBCIndustryCode'],
    parameters={'SDate': 0, 'EDate': -3, 'FRQ': 'FY'}
)
df1

,Instrument,Date,Revenue - Mean,TRBC Economic Sector Name,TRBC Economic Sector Code,TRBC Business Sector Name,TRBC Business Sector Code,TRBC Industry Group Name,TRBC Industry Group Code,TRBC Industry Name,TRBC Industry Code
0,BARC.L,2023-03-06,26185926370,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
1,BARC.L,2023-02-13,25107439220,,,,,,,,
2,BARC.L,2022-02-11,21896182240,,,,,,,,
3,BARC.L,2021-01-28,21603248110,,,,,,,,
4,TRI.N,2023-02-22,6937020650,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070
5,TRI.N,2023-02-01,6626869820,,,,,,,,
6,TRI.N,2022-02-07,6311529500,,,,,,,,
7,TRI.N,2021-02-22,5980789530,,,,,,,,
8,TSLA.O,2023-03-05,103133758370,Consumer Cyclicals,53,Automobiles & Auto Parts,5310,Automobiles & Auto Parts,531010,Auto & Truck Manufacturers,53101010
9,TSLA.O,2023-01-25,81715341140,,,,,,,,


In [52]:
# The below in this cell is needed to forward fill our dataframe correctly:
df1.replace({'': np.nan}, inplace=True)
df1.where(pd.notnull(df1), np.nan, inplace=True)

for i in df1.groupby(by=["Instrument"]):
    if i[0] == df1["Instrument"][0]: _df1 = i[1].ffill()
    else: _df1 = _df1.append(i[1].ffill())
_df1

,Instrument,Date,Revenue - Mean,TRBC Economic Sector Name,TRBC Economic Sector Code,TRBC Business Sector Name,TRBC Business Sector Code,TRBC Industry Group Name,TRBC Industry Group Code,TRBC Industry Name,TRBC Industry Code
0,BARC.L,2023-03-06,26185926370,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
1,BARC.L,2023-02-13,25107439220,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
2,BARC.L,2022-02-11,21896182240,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
3,BARC.L,2021-01-28,21603248110,Financials,55,Banking & Investment Services,5510,Banking Services,551010,Banks,55101010
4,TRI.N,2023-02-22,6937020650,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070
5,TRI.N,2023-02-01,6626869820,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070
6,TRI.N,2022-02-07,6311529500,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070
7,TRI.N,2021-02-22,5980789530,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070
8,TSLA.O,2023-03-05,103133758370,Consumer Cyclicals,53,Automobiles & Auto Parts,5310,Automobiles & Auto Parts,531010,Auto & Truck Manufacturers,53101010
9,TSLA.O,2023-01-25,81715341140,Consumer Cyclicals,53,Automobiles & Auto Parts,5310,Automobiles & Auto Parts,531010,Auto & Truck Manufacturers,53101010


##### Snapshot Requests Tip 1
Some fundamental fields will give multiple rows for a given day - for example if we request ratings sources - there could be more than one per date eg if there are 5 ratings agencies providing a rating - this is not usual for a time series history - or perhaps it is very different say than non-expandable single point timeseries. In this example as we have multiple RICS whose ratings dates may not overlap ie be on the same row `<NA>` artifacts are added to deliver the dataframe

In [55]:
df2 = rd.get_history(
    universe=['BARC.L', 'TRI.N','TSLA.O'],
    fields=['TR.IR.RatingSourceDescription', 'TR.IR.RatingSourceType',
            'TR.IR.Rating','TR.IR.Rating.date'],
    interval="1Y",
    start="2015-01-25",
    end="2022-02-01")

df2

BARC.L                     \
                                Rating Source Description Rating Source Type   
Date                                                                           
2015-07-16                                           <NA>               <NA>   
2015-11-19                         Fitch Senior Unsecured                FSU   
2015-11-19                   Fitch Short-term Debt Rating                FDT   
2016-08-16                                           <NA>               <NA>   
2016-12-12                Moody's Long-term Issuer Rating                MIS   
2016-12-12  Moody's Long-term Senior Unsecured MTN Rating                MMU   
2016-12-12                       Moody's Senior Unsecured                MSU   
2017-04-27                                           <NA>               <NA>   
2017-08-07                                           <NA>               <NA>   
2017-08-07                                           <NA>               <NA>   
2017-08-07                                           <NA>               <NA>   
2017-10-06      S&P Long-term Senior Unsecured MTN Rating                SMU   
2018-08-15                                           <NA>               <NA>   
2018-10-04                                           <NA>               <NA>   
2018-10-04                                           <NA>               <NA>   
2018-12-28                    Egan-Jones Commercial Paper                EJC   
2019-10-25                                           <NA>               <NA>   
2019-11-27                    R&I Long-term Issuer Rating                RII   
2019-12-02                                           <NA>               <NA>   
2020-05-11                                           <NA>               <NA>   
2020-11-18                    R&I Long-term Issuer Rating                RII   
2020-12-17                                           <NA>               <NA>   
2020-12-17                                           <NA>               <NA>   
2021-10-22                                           <NA>               <NA>   
2021-10-22                                           <NA>               <NA>   
2021-11-26                    R&I Long-term Issuer Rating                RII   

                                                                   TRI.N  \
           Issuer Rating       Date            Rating Source Description   
Date                                                                       
2015-07-16          <NA>        NaT                 S&P Senior Unsecured   
2015-11-19             A 2015-11-19                                 <NA>   
2015-11-19            F1 2015-11-19                                 <NA>   
2016-08-16          <NA>        NaT               Fitch Senior Unsecured   
2016-12-12          Baa2 2016-12-12                                 <NA>   
2016-12-12       (P)Baa2 2016-12-12                                 <NA>   
2016-12-12          Baa2 2016-12-12                                 <NA>   
2017-04-27          <NA>        NaT  S&P Short-term Issuer Credit Rating   
2017-08-07          <NA>        NaT                                 <NA>   
2017-08-07          <NA>        NaT                                 <NA>   
2017-08-07          <NA>        NaT                                 <NA>   
2017-10-06           BBB 2017-10-06                                 <NA>   
2018-08-15          <NA>        NaT                                 <NA>   
2018-10-04          <NA>        NaT          S&P Long-term Issuer Rating   
2018-10-04          <NA>        NaT                 S&P Senior Unsecured   
2018-12-28            A1 2018-12-28                                 <NA>   
2019-10-25          <NA>        NaT                                 <NA>   
2019-11-27            A- 2019-11-27                                 <NA>   
2019-12-02          <NA>        NaT          Egan-Jones Senior Unsecured   
2020-05-11          <NA>        NaT             Moody's Se

##### Snapshot Requests Tip 2
Again this same multi-row exanding timeseries history - here in the case of broker recommendations - is another example - with lots of `<NA>` artifacts added. 

In [36]:
df2 = rd.get_history(
    universe=['BARC.L', 'TRI.N', 'TSLA.O'],
    fields=['TR.RecEstValue', 'TR.TPEstValue', 'TR.EPSEstValue'],
    interval="1M",
    start="2020-01-25",
    end="2022-02-01")
df2

BARC.L  \
                    Standard Rec (1-5) - Broker Estimate   
Date                                                       
2013-10-02 00:00:00                                    2   
2013-10-02 00:00:00                                    2   
2015-02-12 00:00:00                                 <NA>   
2015-02-12 00:00:00                                 <NA>   
2015-02-12 00:00:00                                 <NA>   
...                                                  ...   
2022-01-31 00:00:00                                 <NA>   
2022-01-31 04:00:00                                 <NA>   
2022-01-31 07:27:00                                 <NA>   
2022-01-31 21:05:00                                 <NA>   
2022-01-31 23:00:00                                 <NA>   

                                                    \
                    Target Price - Broker Estimate   
Date                                                 
2013-10-02 00:00:00                           <NA>   
2013-10-02 00:00:00                           <NA>   
2015-02-12 00:00:00                           <NA>   
2015-02-12 00:00:00                           <NA>   
2015-02-12 00:00:00                           <NA>   
...                                            ...   
2022-01-31 00:00:00                           <NA>   
2022-01-31 04:00:00                           <NA>   
2022-01-31 07:27:00                          248.3   
2022-01-31 21:05:00                           <NA>   
2022-01-31 23:00:00                           <NA>   

                                                          \
                    Earnings Per Share - Broker Estimate   
Date                                                       
2013-10-02 00:00:00                                 <NA>   
2013-10-02 00:00:00                                 <NA>   
2015-02-12 00:00:00                                 <NA>   
2015-02-12 00:00:00                                 <NA>   
2015-02-12 00:00:00                                 <NA>   
...                                                  ...   
2022-01-31 00:00:00                                 <NA>   
2022-01-31 04:00:00                                 <NA>   
2022-01-31 07:27:00                                 <NA>   
2022-01-31 21:05:00                                 <NA>   
2022-01-31 23:00:00                                 <NA>   

                                                   TRI.N  \
                    Standard Rec (1-5) - Broker Estimate   
Date                                                       
2013-10-02 00:00:00                                 <NA>   
2013-10-02 00:00:00                                 <NA>   
2015-02-12 00:00:00                                 <NA>   
2015-02-12 00:00:00                                 <NA>   
2015-02-12 00:00:00                                 <NA>   
...                                                  ...   
2022-01-31 00:00:00                                 <NA>   
2022-01-31 04:00:00                                 <NA>   
2022-01-31 07:27:00                                 <NA>   
2022-01-31 21:05:00                                 <NA>   
2022-01-31 23:00:00                                 <NA>   

                                                    \
                    Target Price - Broker Estimate   
Date                                                 
2013-10-02 00:00:00                           <NA>   
2013-10-02 00:00:00                           <NA>   
2015-02-12 00:00:00                           <NA>   
2015-02-12 00:00:00                           <NA>   
2015-02-12 00:00:00                           <NA>   
...                                            ...   
2022-01-31 00:00:00                           <NA>   
2022-01-31 04:00:00                           <NA>   
2022-01-31 07:27:00                           <NA>   
2022-01-31 21:05:00                           <NA>   
2022-01-31 23:00:00                           <NA>   

             

## DEX2

### What does this do?

The DEX2 interface provided a broad range of content and functionality such Fundamental and Reference as well as timeseries histories of such fields. The DEX2 interface also provided interday timeseries histories for both realtime and non-realtime data. In addition you could use it for symbology conversion. 

### DEX2 ID to RIC (Symbology Conversion)

As shown in [Tutorial 6 - Data Engine - Dex2](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-6-data-engine-dex-2)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/dex2.zip), we can convert symbols using the COM API.

#### VBA

After setting `MyDex2Mgr = CreateReutersObject("Dex2.Dex2Mgr")`, for ISINs, we can then go ahead with

>     ' Must call Initialise() at once
>     m_cookie = MyDex2Mgr.Initialize()
>     ' We can then create an RData object
>     Set myRData1 = MyDex2Mgr.CreateRData(m_cookie)
>     
>     With MyDex2Mgr
>         .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>     End With
>     
>     ' Set the properties & methods for the DEX2Lib.RData object
>     With myRData1
>         .InstrumentIDList = Range("B7").Value ' Could use "TRI.N", "TRI.N; GOOG.O; MSFT.O" or array, e.g. arrInstrList.
>         .FieldList = Range("D7").Value & "; " & Range("D8").Value  '"RI.ID.RIC; RI.ID.ISIN" ' Could use "RI.ID.RIC", "RI.ID.RIC; RI.ID.WERT" or array, e.g. arrFldList.
>         .DisplayParam = "Transpose:Y"
>         'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>             "", "RH:In CH:Fd Transpose:Y"
>         'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>         
>         'Ignore cache; get data directly from the Snapshot Server
>         '.Subscribe False
>         'Or use cache by default
>         .Subscribe
>     End With
>     Exit Sub

For SEDOL, we can use:

>     ' Must call Initialise() at once
>     m_cookie = MyDex2Mgr.Initialize()
>     ' We can then create an RData object
>     Set myRData2 = MyDex2Mgr.CreateRData(m_cookie)
>     
>     With MyDex2Mgr
>         .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>     End With
>     
>     ' Set the properties & methods for the DEX2Lib.RData object
>     With myRData2
>         .InstrumentIDList = [B10].Value ' Could use "TRI.N", "TRI.N; GOOG.O; MSFT.O" or array, e.g. arrInstrList.
>         .FieldList = Range("D10").Value & "; " & Range("D11").Value  '"RI.ID.RIC; RI.ID.SEDOL" ' Could use "RI.ID.RIC", "RI.ID.RIC; RI.ID.WERT" or array, e.g. arrFldList.
>         .DisplayParam = "Transpose:Y"
>         'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>             "", "RH:In CH:Fd Transpose:Y"
>         'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>         
>         'Ignore cache; get data directly from the Snapshot Server
>         '.Subscribe False
>         'Or use cache by default
>         .Subscribe
>     End With
>     Exit Sub

#### Python

Symbology conversions in python are a lot more powerful than in Excel using the COM API. The extent of the use-cases for symbology conversions are best explained in our [GitHub Repository](https://github.com/Refinitiv-API-Samples/Example.DataLibrary.Python/blob/main/Examples/2-Content/2.05-Symbology/EX-2.05.01-Symbology.ipynb), but for the sake of completeness, you can see an example use in the 2 cells directly below:

In [45]:
# # Get generic table of many symbols for 4 companies of choice:
response = rd.content.symbol_conversion.Definition(symbols=["MSFT.O", "AAPL.O", "GOOG.O", "IBM.N"]).get_data()
response.data.df

,DocumentTitle,RIC,IssueISIN,CUSIP,SEDOL,TickerSymbol,IssuerOAPermID
MSFT.O,"Microsoft Corp, Ordinary Share, NASDAQ Global ...",MSFT.O,US5949181045,594918104,<NA>,MSFT,4295907168
AAPL.O,"Apple Inc, Ordinary Share, NASDAQ Global Selec...",AAPL.O,US0378331005,037833100,<NA>,AAPL,4295905573
GOOG.O,"Alphabet Inc, Ordinary Share, Class C, NASDAQ ...",GOOG.O,US02079K1079,02079K107,<NA>,GOOG,5030853586
IBM.N,"International Business Machines Corp, Ordinary...",IBM.N,US4592001014,459200101,2005973,IBM,4295904307


In [46]:
# # Convert specific symbols:
response = rd.content.symbol_conversion.Definition(
    symbols=["US5949181045", "US02079K1079"],
    from_symbol_type=rd.content.symbol_conversion.SymbolTypes.ISIN,  # Symbol types: RIC => RIC; ISIN => IssueISIN; CUSIP => CUSIP; SEDOL => SEDOL; TICKER_SYMBOL => TickerSymbol; OA_PERM_ID => IssuerOAPermID; LIPPER_ID => FundClassLipperID
    to_symbol_types=[rd.content.symbol_conversion.SymbolTypes.RIC,
                     rd.content.symbol_conversion.SymbolTypes.OA_PERM_ID],
).get_data()

response.data.df

,DocumentTitle,RIC,IssuerOAPermID
US5949181045,"Microsoft Corp, Ordinary Share, NASDAQ Global ...",MSFT.O,4295907168
US02079K1079,"Alphabet Inc, Ordinary Share, Class C, NASDAQ ...",GOOG.O,5030853586


### DEX2 RData Equity Data

As best exemplified in [Tutorial 6 - Data Engine - Dex2](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-6-data-engine-dex-2)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/dex2.zip), the DEX2 RData Equity Data allows us to access historical [Market Capital](https://www.investopedia.com/terms/m/marketcapitalization.asp):

<img src="imgs/Dex2Equity.PNG">
# take this generic stuff down into appendix and make sure all components are referenced

#### VBA

DEX2 replicates RData() and TR() functions for real time data retrieval in Eikon for Excel / Eikon - Microsoft Office. The DEX2 methods differ slightly in the initialisation and use of the Dex2Mgr object depending upon whether RData() or the TR() function is being replicated. The examples below explain this. The TR() function can be used to retrieve data from ADC (the Analytics Data Cloud) and this is contained in Tutorial 11 - Dex2 TR function Analytics Data Cloud ADC data here.

NOTE - Eikon for Excel or Eikon - Microsoft Office should be logged in otherwise the sample VBA code will generate an error "ERROR #360c - AdxRtx : No connection to the platform".

##### Prerequisites
1. Open a new single sheet Excel workbook. Save As with an appropriate name (e.g. DEX2.xls or DEX2.xlsm in Office 2007 or higher).
2. Go to the VBE (Visual Basic Editor), ensure the Project Explorer is visible and select the project for the workbook above. \<ALT\>\<F11\> or Tools, Macro, Visual Basic Editor in Excel 2003 or  Developer, Visual Basic in Excel 2007 and above, View, Project Explorer If the Developer header is not visible in Excel 2007 and above, go to the Excel Office Button, select Excel Options (lower right), Popular, and check the 'Show Developer tab in the Ribbon' box.
3. In the VBE, click on File, Import File and import PLVbaApis.bas. The .bas location is C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin (Z may be X or Y, depending on the last Eikon update). The .bas is loaded as a new VB project module, PLVbaApis.
4. In the PLVbaAPis module, comment out the sections which aren't required. As this example is dealing with DEX2, part of the library DEX2Lib, the DEX2 section can remain uncommented.
5. In the VBE, go Tools, References and ensure that the DEX2 Library is checked. If it is not in the list the library is called DEX2.dll and its location for Eikon 4 is;

    "C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin" (Z may be X or Y, depending on the last Eikon update).

    Note that for the latest versions of Eikon the default path is;

    "C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin\Apps\TR.OFFICE.CORE\0.0.0.0\Bin" (Z may be X or Y, depending on the last Eikon update).

##### DEX2

6. Create an instance of an DEX2Mgr object using the PLVbaApis function CreateDex2Mgr().

    Set myDEX2Mgr = CreateDex2Mgr()
 
7. To replicate RData(), create the Dex2Mgr object (myDex2Mgr) and initialise it. Use the return value from the initialisation to create an instance of the Dex2Lib.RData Class (myDex2RData) using the CreateRData method.

8. Now go ahead with `cmdMktCapHist_Click` creation:

>    Private Sub cmdMktCapHist_Click()
>        On Error GoTo errHandler
>        
>        ActiveCell.Select
>        
>        Range("F20:G100").ClearContents
>        If MyDex2Mgr Is Nothing Then Set MyDex2Mgr = CreateReutersObject("Dex2.Dex2Mgr")
>        
>        ' Must call Initialise() at once
>        m_cookie = MyDex2Mgr.Initialize()
>        ' We can then create an RData object
>        Set myRData2 = MyDex2Mgr.CreateRData(m_cookie)
>        
>        With MyDex2Mgr
>            .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>        End With
>        
>        ' Set the properties & methods for the DEX2Lib.RData object.
>        With myRData2
>            .InstrumentIDList = [C15].Value
>            .FieldList = [C16].Value '"RII.A.INDEX_MKT_CAP_RTRS" '"RII.A.INDEX_MKT_CAP_RTRS; RII.A.INDEX_MKT_CAP_USD_RTRS"
>            ' Could use single "RI.ID.RIC", multiple "RI.ID.RIC; RI.ID.WERT" or array, e.g. arrFldList.
>            .RequestParam = [C17].Value
>            .DisplayParam = "RH:D CH:Fd SORT:DESC"
>            'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>                "", "RH:In CH:Fd Transpose:Y"
>            'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>            
>            'Ignore cache; get data directly from the Snapshot Server
>            '.Subscribe False
>            'Or use cache by default
>            .Subscribe
>        End With
>        Exit Sub
>        
>    errHandler:
>        MsgBox MyDex2Mgr.GetErrorString(Err.Number)
>    End Sub
>    
>    ' OnUpdate event callback for myRData1
>    Private Sub myRData2_OnUpdate(ByVal DataStatus As Dex2Lib.DEX2_DataStatus, ByVal Error As Variant)
>        Dim C As Integer, r As Integer
>        Dim res2 As Variant
>        
>        If Error <> 0 Then [F21].Value = Error: Exit Sub
>    
>        res2 = myRData2.Data
>        
>        If IsEmpty(res2) Then [F21].Value = "No data": Exit Sub
>        
>        ' r for the rows
>        For r = LBound(res2, 1) To UBound(res2, 1)
>            ' c for the columns
>            For C = LBound(res2, 2) To UBound(res2, 2)
>                [F21].Offset(r, C).Value = res2(r, C)
>            Next C
>        Next r
>    End Sub

#### Python

Couldn't be easier in Python:

In [47]:
MarketCapDf = rd.get_history(
    universe=['VOD.L','BARC.L'],
    fields=['TR.CompanyMarketCapitalization'],
    interval="1D", # The consolidation interval. Supported intervals are: tick, tas, taq, minute, 1min, 5min, 10min, 30min, 60min, hourly, 1h, daily, 1d, 1D, 7D, 7d, weekly, 1W, monthly, 1M, quarterly, 3M, 6M, yearly, 1Y.
    start="2022-08-10",
    end="2022-09-13")
MarketCapDf

Company Market Capitalization,VOD.L,BARC.L
Date,,
2022-08-10,33802223010.1754,27530345491.2761
2022-08-11,33794859479.455799,27383302388.1096
2022-08-12,33887838289.451801,27786941607.566399
2022-08-15,33760332255.312,27758457269.677299
2022-08-16,34330055002.498199,27582105369.2075
2022-08-17,33921476881.010399,27285640404.213501
2022-08-18,33852812314.98,27254991056.654099
2022-08-19,33895552270.749599,26715133223.987099
2022-08-22,33370390822.183201,26340483067.0546


### DEX2 Time Series

DEX2 returns reference and fundamental data for instruments. Data includes identifiers such as ISIN and Cusip codes for an instrument, market capitalisation history, business sector information, ratings information and time series data. This was best exemplified in [Tutorial 6 - Data Engine - Dex2](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-6-data-engine-dex-2)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/dex2.zip):

<img src="imgs/DEX2 TR Data.PNG">

#### VBA

You'd have to Initialize Dex2 session under the sub `cmdInitialize_Click()`:

>    ' Initialize Dex2 session when the data request is first started.
>    Private Sub cmdInitialize_Click()
>        On Error GoTo ErrorHandle
>    
>        ActiveCell.Select
>    
>        ' Clear the output of logger
>        lbLog.Clear
>    
>        ClearAllOutput
>    
>        ' Clear the Cookie and old query
>        If Not MyDex2Cookie = 0 Then
>            MyDex2Mgr.Finalize (MyDex2Cookie)
>            MyDex2Cookie = 0
>            Set MyDex2Mgr = Nothing
>            Set MyDex2RData = Nothing
>        End If
>        
>        ' Instantiate the Dex2 manager
>        Set MyDex2Mgr = CreateDex2Mgr()
>        Set MyDex2MgrADC = MyDex2Mgr
>        ' Instantiate the RSearch logger
>        Set MyDex2Logger = New CLogger
>        
>        ' Initialize Dex2 session
>        MyDex2Cookie = MyDex2MgrADC.Initialize(DE_MC_ADC_POWERLINK, MyDex2Logger)
>            
>        ' We can choose to display error code
>        MyDex2Mgr.SetErrorHandling MyDex2Cookie, DE_EH_ERROR_CODES
>        ' Or display error description
>        ' MyDex2Mgr.SetErrorHandling MyDex2Cookie, DE_EH_STRING
>            
>        ' Create a Dex2 query using the session cookie
>        Set MyDex2RData = MyDex2Mgr.CreateRData(MyDex2Cookie)
>        
>        ' Create a Dex2 query manager using the session cookie
>        Set MyDex2RDataMgr = MyDex2Mgr.CreateRDataMgr(MyDex2Cookie)
>        
>        Exit Sub

#### Python

In Python, things are a little simpler:

In [48]:
DEX2Df = rd.get_history(
    universe=['VOD.L', 'BARC.L', 'TRI.N'],
    fields=['TR.TotalReturn', 'TR.PRICECLOSE'],
    interval="1D",
    start="2021-10-01",
    end="2021-10-11")

DEX2Df

VOD.L                   BARC.L                    TRI.N  \
           Total Return Price Close Total Return Price Close Total Return   
Date                                                                        
2021-10-01    -0.970874       112.2    -0.907173      187.88    -0.289645   
2021-10-04       1.4082      113.78    -0.319353      187.28     -0.67175   
2021-10-05     0.773422      114.66     3.908586       194.6     0.923049   
2021-10-06    -2.651317      111.62    -1.387461       191.9     1.213438   
2021-10-07     0.734635      112.44     0.958833      193.74     1.673079   
2021-10-08    -0.302383       112.1     0.732941      195.16     0.228793   
2021-10-11    -0.303301      111.76     1.291248      197.68     0.412643   

                        
           Price Close  
Date                    
2021-10-01      110.16  
2021-10-04      109.42  
2021-10-05      110.43  
2021-10-06      111.77  
2021-10-07      113.64  
2021-10-08       113.9  
2021-10-11      114.37

### DEX2 RData Treasury Data

This functionality was best shown in [Tutorial 6 - DEX2 Treasury](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-6-data-engine-dex-2) where Treasury Ratings and their Source Descriptions were discoverable:

<img src="imgs/DEX2 RData Treasury Data.PNG">

#### VBA

>     Private Sub cmdClearAll_Click()
>         ActiveCell.Select
>         
>         Range("G10").Value = ""
>         Range("G21:J100").Value = ""
>         
>         If Not myRData1 Is Nothing Then Set myRData1 = Nothing
>         If Not myRData2 Is Nothing Then Set myRData2 = Nothing
>         If Not MyDex2Mgr Is Nothing Then Set MyDex2Mgr = Nothing
>     End Sub
>     
>     Private Sub cmdGetRating_Click()
>         On Error GoTo errHandler
>         
>         ActiveCell.Select
>         
>         [G7].Value = ""
>         ' Note the use of CreateReutersObject - function in the PLVbaApis module.
>         If MyDex2Mgr Is Nothing Then Set MyDex2Mgr = CreateReutersObject("Dex2.Dex2Mgr")
>         
>         ' Must call Initialise() at once
>         m_cookie = MyDex2Mgr.Initialize()
>         ' We can then create an RData object
>         Set myRData1 = MyDex2Mgr.CreateRData(m_cookie)
>         
>         With MyDex2Mgr
>             .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>         End With
>         
>         ' Set the properties & methods for the DEX2Lib.RData object
>         With myRData1
>             .InstrumentIdList = [C6].Value ' Could use "TRI.N", "TRI.N; GOOG.O; MSFT.O" or array, e.g. arrInstrList.
>             .FieldList = [C7].Value '"EJV.GR.Rating" ' Could use "RI.ID.RIC", "RI.ID.RIC; RI.ID.WERT" or array, e.g. arrFldList.
>             .RequestParam = [C8].Value '"RTSRC:S&P"
>             'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>                 "", "RH:In CH:Fd Transpose:Y"
>             'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>             
>             'Ignore cache; get data directly from the Snapshot Server
>             '.Subscribe False
>             'Or use cache by default
>             .Subscribe
>         End With
>         Exit Sub
>         
>     errHandler:
>         MsgBox MyDex2Mgr.GetErrorString(Err.Number)
>     End Sub
>     
>     ' OnUpdate event callback for myRData1
>     Private Sub myRData1_OnUpdate(ByVal DataStatus As Dex2Lib.DEX2_DataStatus, ByVal Error As Variant)
>         Dim res As Variant
>     
>         'Debug.Print DataStatus
>     
>         If Error <> 0 Then [G10].Value = Error: Exit Sub
>     
>         ' get the data retrieved from the database
>         res = myRData1.Data
>     
>         ' Display the result.
>         [G10].Value = res
>     End Sub
>     
>     Private Sub cmdGetHistoryOfRating_Click()
>         On Error GoTo errHandler
>         
>         ActiveCell.Select
>         
>         Range("G21:J100").ClearContents
>         If MyDex2Mgr Is Nothing Then Set MyDex2Mgr = CreateReutersObject("Dex2.Dex2Mgr")
>         
>         ' Must call Initialise() at once
>         m_cookie = MyDex2Mgr.Initialize()
>         ' We can then create an RData object
>         Set myRData2 = MyDex2Mgr.CreateRData(m_cookie)
>         
>         With MyDex2Mgr
>             .SetErrorHandling m_cookie, DE_EH_STRING ' Could also use DE_EH_ERROR_CODES
>         End With
>         
>         ' Set the properties & methods for the DEX2Lib.RData object
>         With myRData2
>             .InstrumentIdList = [C15].Value
>             .FieldList = [C16].Value
>             .RequestParam = [C17].Value
>             .DisplayParam = [C18].Value
>             
>             'N.B. Could use .SetParameter "TRI.N; MSFT.O","RF.G.COMPNAME; RF.G.NUMEMPLOY; RF.G.CNTEMAIL", _
>                 "", "RH:In CH:Fd Transpose:Y"
>             'Hence .SetParameter [InstrumentIDList].Value, [FieldList].Value, [RequestParam].Value, [DisplayParam].Value
>             
>             'Ignore cache; get data directly from the Snapshot Server
>             '.Subscribe False
>             'Or use cache by default
>             .Subscribe
>         End With
>         Exit Sub
>         
>     errHandler:
>         MsgBox MyDex2Mgr.GetErrorString(Err.Number)
>     End Sub


#### Python

Things are much simpler in Python:

In [50]:
rating = rd.get_history(
    universe=['GB137283921='],
    fields=['TR.IR.RatingSourceDescription', 'TR.IR.RatingSourceType',
            'TR.IR.Rating'],  
    interval="1Y",
    start="2015-01-25",
    end="2023-02-01")
rating

GB137283921=,Rating Source Description,Rating Source Type,Issuer Rating
Date,,,
2015-12-17,Egan-Jones Commercial Paper,EJC,A1
2015-12-17,Egan-Jones Senior Unsecured,EJU,BBB+
2016-07-06,Egan-Jones Senior Unsecured,EJU,BBB
2018-10-04,Fitch Subordinated,FBD,BBB-
2019-11-26,Egan-Jones Senior Unsecured,EJU,BBB-
2021-07-29,Fitch Commercial Paper,FCP,F2
2022-11-25,Egan-Jones Commercial Paper,EJC,A1


### DEX2 Fundamental and Reference

<img src="imgs/Dex2 RData Fundamental Data.PNG">

Time Series data as the name suggests changes with time. Reference data, on the other hand, does not primarily change with time; when it does, it usually accounts for a structural change in the instrument at play (e.g.: a company changes from banking to tech operation industries or is acquired and changes name etc). Fundamental data is released temporally say quarterly or half-yearly or annually and is usually looked at across time eg Revenue or Sales. DEX2 provides access to all of our TR fields - which allow you to specify begin dates and end dates amongst other parameters. Access to these is unchanged in our new APIs - only its a lot simpler.

#### VBA

In VBA, after creating '`Private MyDex2Mgr As Dex2Lib.Dex2Mgr`':

>     Private MyDex2Mgr As Dex2Lib.Dex2Mgr
>     Private MyDex2MgrADC As Dex2Lib.IDex2Mgr2
>     Private MyDex2Cookie As Long
>     ' Private variable that holds the Dex2 RData, able to obtain TR fields.
>     Private WithEvents MyDex2RData As Dex2Lib.RData

to complete the Tutorial 6 example, one could go through the following to collect DEX2 Fundamental and Reference data:

>         ' Now create the Dex2 and IDex2 objects.
>         Set MyDex2Mgr = CreateDex2Mgr()
>         Set MyDex2MgrADC = MyDex2Mgr
>         
>         ' Initialise using the DE_MC_ADC_POWERLINK enumeration
>         MyDex2Cookie = MyDex2MgrADC.Initialize(DE_MC_ADC_POWERLINK)
>         ' And create an RData object.
>         Set MyDex2RData = MyDex2Mgr.CreateRData(MyDex2Cookie)
>         
>         With MyDex2RData
>             .InstrumentIDList = Range("G6").Value ' Or for multiple isntruments "TRI.N;MSFT.O;GOOG.O"
>             .FieldList = strInstrList '  Or single field "TR.CLOSE"
>             .RequestParam = "" ' Or of the form "edate:-20d sdate:-9d"
>             .DisplayParam = "CH:Fd" ' Or of the form "RH:In CH:Fd"
>             
>             '' OR can use .SetParameter
>             '.SetParameter Range("G6").Value, strInstrList, "", "CH:Fd"
>             
>             ' Send the query without using cache
>             .Subscribe False  ' Or use cache by default - myDex2RData.Subscribe
>             
>             ' When the data is returned, the myDex2RData_OnUpdate event and Sub are 'fired'.

#### Python

Fundamental data was covered above under 'RData Fundamentals'; such data can be found in Python with [RD](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) (and `rd.get_data`, `rd.get_history` or `rd.content.fundamental_and_reference.Definition`) or [EDAPI](https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api).

#### Fundamental and Reference Timeseries

##### VBA

DEX2 Fundamental and Reference Timeseries data could be fetched similarly to simple time series data:

Start with 

>     ' Private Variable that holds the instance of the Dex2 manager singleton
>     Private MyDex2Mgr As Dex2Lib.Dex2Mgr
>     Private MyDex2MgrADC As Dex2Lib.IDex2Mgr2
>     
>     ' Private variable that holds the cookie that identifies the Dex2 session
>     Private MyDex2Cookie As Long
>     ' Private variable that holds the Dex2 RData
>     Private WithEvents MyDex2RData As Dex2Lib.RData
>     ' Private variable that holds the Dex2 RDataMgr
>     Private MyDex2RDataMgr As Dex2Lib.RDataMgr
>     
>     ' Private variable that holds the Dex2 logger
>     Private MyDex2Logger As CLogger

Then initialise clicks:

>         ' Instantiate the Dex2 manager
>         Set MyDex2Mgr = CreateDex2Mgr()
>         Set MyDex2MgrADC = MyDex2Mgr
>         ' Instantiate the RSearch logger
>         Set MyDex2Logger = New CLogger
>         
>         ' Initialize Dex2 session
>         MyDex2Cookie = MyDex2MgrADC.Initialize(DE_MC_ADC_POWERLINK, MyDex2Logger)
>             
>         ' We can choose to display error code
>         MyDex2Mgr.SetErrorHandling MyDex2Cookie, DE_EH_ERROR_CODES
>         ' Or display error description
>         ' MyDex2Mgr.SetErrorHandling MyDex2Cookie, DE_EH_STRING
>             
>         ' Create a Dex2 query using the session cookie
>         Set MyDex2RData = MyDex2Mgr.CreateRData(MyDex2Cookie)
>         
>         ' Create a Dex2 query manager using the session cookie
>         Set MyDex2RDataMgr = MyDex2Mgr.CreateRDataMgr(MyDex2Cookie)
   
Then, after some error & event handling:

>         ' Set input values
>                 MyDex2RData.SetParameter _
>                     Range("Dex2Item").Value, _
>                     Range("Dex2Fields").Value, _
>                     Range("Dex2RequestParameters").Value, _
>                     Range("Dex2DisplayParameters").Value
>                     ' Or using the following individual properties
>                     ' MyDex2RData.InstrumentIDList = Range("Dex2Item").Value
>                     ' MyDex2RData.FieldList = Range("Dex2Fields").Value
>                     ' MyDex2RData.requestParam = Range("Dex2RequestParameters").Value
>                     ' MyDex2RData.displayParam = Range("Dex2DisplayParameters").Value
>     
>                 ' Send the query without using cache
>                 MyDex2RData.Subscribe (False)
>                 ' Or use cache by default
>                 ' MyDex2RData.Subscribe

Before creating 'OnUpdate' code.

This is all quite heavy in VBA, while it could hardly be simpler in Python:


##### Python

This was covered above, in the section 'RData Fundamentals' using [RD](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python)'s `rd.get_history` function. We can also do this with the `rd.get_data` function:

In [60]:
DEX2TrDf = rd.get_data(
    ['VOD.L', 'MSFT.N', 'TRI.N'],
    fields=[
        'TR.RevenueMean.date', 'TR.RevenueMean', 'TR.TRBCEconomicSector', 'TR.TRBCEconSectorCode',
        'TR.TRBCBusinessSector', 'TR.TRBCBusinessSectorCode', 'TR.TRBCIndustryGroup',
        'TR.TRBCIndustryGroupCode', 'TR.TRBCIndustry', 'TR.TRBCIndustryCode'],
    parameters={'SDate': 0, 'EDate': -3, 'FRQ': 'FY'})

DEX2TrDf

,Instrument,Date,Revenue - Mean,TRBC Economic Sector Name,TRBC Economic Sector Code,TRBC Business Sector Name,TRBC Business Sector Code,TRBC Industry Group Name,TRBC Industry Group Code,TRBC Industry Name,TRBC Industry Code
0,VOD.L,2023-02-14,45864975420,Technology,57,Telecommunications Services,5740,Telecommunications Services,574010,Wireless Telecommunications Services,57401020
1,VOD.L,2022-05-17,45443437460,,,,,,,,
2,VOD.L,2021-05-11,43500994090,,,,,,,,
3,VOD.L,2020-05-07,45001539900,,,,,,,,
4,MSFT.N,2023-03-01,208894168150,Technology,57,Software & IT Services,5720,Software & IT Services,572010,Software,57201020
5,MSFT.N,2022-07-25,198837734950,,,,,,,,
6,MSFT.N,2021-07-26,166243106170,,,,,,,,
7,MSFT.N,2020-07-20,141503225060,,,,,,,,
8,TRI.N,2023-02-22,6937020650,Industrials,52,Industrial & Commercial Services,5220,Professional & Commercial Services,522030,Professional Information Services,52203070
9,TRI.N,2023-02-01,6626869820,,,,,,,,


## RHistory

### What does this do?

As per the [EIKON FOR DEVELOPERS](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/documentation/eikon_for_developers_09032016.pdf) documentation that you may find [here](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/documentation),

    The RHistory API enables developers to access time series data in VBA inside Excel in the same way as the RHistory function. The RHistory function retrieves a list of time series data for one instrument or a list of instruments at regular intervals (for example, on a daily, weekly, monthly, and yearly basis) for a given time period or for a given number of records. It also provides time series data at non-regular intervals, for example, TAS (Time and Sales), TAQ (Trade and Quote), and TICK (tick by tick).
    
    
This COM API is best exemplified in [Tutorial 9 - Time Series History - RHistoryAPI](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-9-time-series-history-r-history-api)'s [Excxel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/rhistoryapi.zip) in which you will find the following VBA:
    
### VBA

In VBA, we declared `myRHistoryQuery` via `Private WithEvents myRHistoryQuery As RHistoryAPI.RHistoryQuery` before:

>         ' Set the query parameters accordingly to your needs
>         With myRHistoryQuery
>             .InstrumentIdList = Range("G6").Value
>             .FieldList = Range("G7").Value 'Or of the form  "TRDPRC_1.TIMESTAMP;TRDPRC_1.VALUE;TRDPRC_1.VOLUME"
>             
>             '.FieldList = "TRDPRC_1.TIMESTAMP;TRDPRC_1.HIGH;TRDPRC_1.CLOSE;TRDPRC_1.LOW;TRDPRC_1.OPEN;TRDPRC_1.VOLUME;TRDPRC_1.COUNT"
>             .RequestParams = Range("G8").Value
>             .RefreshParams = Range("G9").Value
>             .DisplayParams = Range("G10").Value
>         
>             .Subscribe
>         End With

This allowed us to run sheets such as:

<img src="imgs/RHistory Eg.PNG">


### Python

For this, we can (yet again) use the `get_history` function:

In [44]:
IntradayTimeSeriesDf2 = rd.get_history(
    universe=FTSEConstituents,
    fields=['TRDPRC_1'],
    interval="10min", # The consolidation interval. Supported intervals are: tick, tas, taq, minute, 1min, 5min, 10min, 30min, 60min, hourly, 1h, daily, 1d, 1D, 7D, 7d, weekly, 1W, monthly, 1M, quarterly, 3M, 6M, yearly, 1Y.
    start="2022-06-01T13:00:00",
    end="2022-06-01T16:00:00")

IntradayTimeSeriesDf2

TRDPRC_1,STAN.L,CRDA.L,ANTO.L,BNZL.L,SGE.L,SVT.L,BLND.L,ICAG.L,REL.L,SMIN.L,AZN.L,HSBA.L,CTEC.L,WPP.L,FRES.L,AAF.L,SGRO.L,SJP.L,TW.L,AHT.L,HLMA.L,III.L,CNA.L,MNG.L,BKGH.L,SMDS.L,NG.L,RKT.L,SKG.L,WEIR.L,MRON.L,HSX.L,CPG.L,AUTOA.L,AV.L,ENT.L,DGE.L,INF.L,UU.L,PSHP.L,HLN.L,WTB.L,PRU.L,IMB.L,EXPN.L,BRBY.L,RS1R.L,ABDN.L,GSK.L,LAND.L,BEZG.L,BP.L,JD.L,ABF.L,AAL.L,ADML.L,RTO.L,RMV.L,SBRY.L,PHNX.L,FLTRF.L,IHG.L,BT.L,MNDI.L,BATS.L,PSON.L,CRH.L,SPX.L,PSN.L,RIO.L,JMAT.L,CCH.L,RR.L,SN.L,BMEB.L,SSE.L,SMT.L,FRAS.L,HRGV.L,KGF.L,LLOY.L,NWG.L,SDR.L,NXT.L,ITRK.L,BDEV.L,SHEL.L,GLEN.L,VOD.L,BARC.L,FCIT.L,UTG.L,BAES.L,DCC.L,ULVR.L,EDV.L,OCDO.L,LSEG.L,TSCO.L,LGEN.L
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-06-01 13:00:00,637.6,6918,1509.5,2789,653.2,2856.0,526.8,128.86,2250.0,1572.0,10494,535.6,218.033,937.8,770.0,156.1,1094.0,1260.0,131.0,4125.0,2218.0,1252.5,80.06,218.1,4260,307.4,1128.0,6244,3234,1625.0,135.5,925.6,1814.0,590.4,429.9,1473.5,3669.7108,545.4,1048.5,2585.0,<NA>,2723.0,1024.5,1808.5,2640.0,1735.5,973.0,194.55,1746.19605,770.6,488.0,434.1,123.8,1730.0,3868.0,2171.934128,502.2,592.6,229.7,637.2,9468,4936,189.719,1548.5,3562.5,759.2,3276.5,10560,2192.212,5789.8681,2142.0,1733.0,89.35,1296.5,377.135793,1784.5,816.0,701.0,848.8,264.5,45.515,231.308025,498.78,6504.0,4635,505.8,2373.26,515.582209,127.32,171.2,840.0,1132,779.153,5614,3762.0,1793,925.4,7262.0,260.64,258.4002
2022-06-01 13:10:00,636.6,6936,1511.5,2807,654.4,2852.0,526.4,129.16,2257.0,1573.0,10504,534.0,218.6,938.4,770.2,155.9,1096.5,1259.5,130.9,4144.0,2223.8,1251.5,79.94,217.4,4255,308.2,1126.0,6238,3237,1639.89,135.7,922.0,1815.5,593.2,429.1,1477.9317,3675.0,545.4,1048.5,2585.0,<NA>,2729.0,1027.5,1801.0,2628.0,1741.5,976.5,194.45,1746.19605,770.0,487.4,434.35,124.0,1727.0,3867.0,2170.088567,502.489,595.4,229.2119,636.0,9492,4945,189.2,1549.0,3549.0,757.4,3277.0,10600,2188.678,5790.0,2141.0,1731.5,89.59,1298.5,377.900387,1781.0,818.841,700.0,848.0,264.4,45.465,231.006581,498.1,6512.0,4639,504.6,2373.5,514.219214,127.2,170.48,842.181,1129,777.2,5620,3754.5,1796,930.2,7258.0,260.1,257.7
2022-06-01 13:20:00,636.0,6940,1511.5,2812,655.6,2855.65,526.0,129.02,2255.0,1574.0,10530,534.1,218.4,938.2,770.4,156.1,1098.5,1259.5,130.75,4140.6617,2225.0,1256.5,80.1,217.5,4248,307.6,1128.0,6252,3226,1631.5,135.5,921.8,1816.0,593.8,429.3,1476.0,3672.5,545.4,1048.5,2582.105,<NA>,2727.0,1030.0,1801.0,2627.0,1741.8765,975.0,194.45,1748.216175,771.4,485.4,434.85,124.05,1726.0,3875.0,2172.905476,502.0,597.8,229.41,636.2,9484,4942,189.1,1546.0,3545.0,757.6,3276.5,10615,2192.0,5813.0,2142.0,1728.0,89.68,1298.0,378.091535,1786.0,819.027,700.5,848.8,264.4,45.415,230.303211,498.44,6508.8,4643,504.6,2374.5,515.974212,127.16,170.48,841.4475,1136,776.352,5616,3754.0,1798,935.0,7256.0,260.2,257.6
2022-06-01 13:30:00,635.2,6934,1513.0,2809,658.0,2854.0,524.6,128.54,2252.0,1574.5,10520,533.1,217.6,936.8,769.6,156.2,1097.0,1258.5,130.35,4145.0,2225.0,1252.0,80.04,217.8,4240,307.0,1127.5,6252,3227,1630.0,135.2,920.2,1814.0,593.4,428.9,1473.0,3667.0,542.0,1047.5,2585.0,<NA>,2713.0,1030.0,1799.7495,2624.0,1741.5,974.5,194.3,1742.96385,770.0,485.0,435.25,124.6,1730.0,3880.5,2169.020084,502.4,597.6,229.3,636.0,9484,4923,189.0,1545.5,3537.0,756.0,3275.0,10625,2189.0,5801.0,2137.0,1727.5,89.3795,1294.0,377.135793,1784.5,818.8,699.0,850.4,264.2,45.28,230.102249,498.78,6514.0,4646,502.8,2377.5,516.464216,126.9,169.66,842.0,1132,775.2,5610,3765.5,1807,938.2,7266.0,260.2,257.1
2022-06-01 13:40:00,635.7,6936,1516.0,2810,658.0,2852.0,524.4,128.26,2251.0,1575.0,10516,533.5,218.0,935.8,770.4,155.4,1094.5,1259.5,130.25,4147.0,2224.0,1252.5,80.04,218.1,4237,306.7,1125.7,6244,3232,1636.5,135.15,918.8,1811.0,592.8,429.0,1477.5,3658.0,541.6,1048.5,2590.0,<NA>,2710.0,1031.0,1795.5,2630.0,1737.0,975.5,194.75,1739.529638,769.6,483.6,436.4,125.196,1728.0,3883.0,2169.020084,502.6,597.2,229.5,635.6,9490,4916,189.1457,1544.5,3528.0,756.8,3273.0,10645,2184.0,5806.0

## PLSynchronization Manager

This synchronisation manager was an old method whereby the COM API managed connections to Refinitiv Datasets. It is no longer required in python.

## RSearch

RSearch is a COM API function for instrument level search - designed for building instrument lists conforming to various search criteria. See the example below:


<img src="imgs/RSearch.PNG">

[RSearch is a powerful COM API function](https://my.refinitiv.com/content/mytr/en/policies/training-portal/videoarticle.v11535.html). It was best exemplified in [Tutorial 7 - Instrument Search - RSearch
](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials#tutorial-7-instrument-search-r-search)'s [Excel Workbook](https://developers.refinitiv.com/content/dam/devportal/api-families/eikon/com-apis-for-use-in-microsoft-office/tutorials/rsearch.zip).

### VBA

Creating the sub `cmdRSearch_Click`:

>         ' Instantiate the RSearch manager
>         Set myRSrchMgr = CreateRSearchMgr()
>         
>         If Not myRSrchMgr Is Nothing Then
>             ' Initialize RSearch session (we do not provide any logger here)
>             ' NOTE - This will error if Eikon for Excel is not logged in.
>             myRSrchCookie = myRSrchMgr.Initialize(RS_CT_EIKON)
>             ' Create a RSearch query using the session cookie
>             Set myRSrchQry = myRSrchMgr.CreateRSearchQuery(myRSrchCookie)
>             
>             If Not myRSrchMgr Is Nothing Then
>                 With myRSrchQry
>                     'Initialize the RSearch query with the criteria to use for the search
>                     .AssetClass = [AssetClass].Value
>                     .SearchCriteria = [SearchCriteria].Value '"EPS:>5 RCSIssuerCountryLeaf:Canada"
>                     .SearchParameters = [SearchParameters].Value '"NBROWS:50 SORT:EPS:A"
>                     
>                     ' Send the query
>                     .Send
>                 End With
>             End If
>         End If

Then you can check the status of the query with '`myRSrchQry_OnUpdate`'.

### Python

[RSearch is a powerful COM API function](https://my.refinitiv.com/content/mytr/en/policies/training-portal/videoarticle.v11535.html), and we're happy to announce that the Python equivalent - Search API - is even more powerful. For the full works on it, please do read Nick Zicone's article '[Building Search into your Application Workflow](https://developers.refinitiv.com/en/article-catalog/article/building-search-into-your-application-workflow)'.


The code below is a simple example of how one may use the Search API in Python. It requests M&A data using the filters specified above and orders the data by the announcement date in descending order. More on how you can use search, including guidance, examples, and tips to determine the possible approaches, from simple discovery through experimentation to more advanced techniques, are presented in [this article](https://developers.refinitiv.com/en/article-catalog/article/building-search-into-your-application-workflow).

In [64]:
bondsSerarch = search.Definition(
    view=search.Views.BOND_FUT_OPT_QUOTES,  # for info on `SearchViews`, you can use `help(search.Views)`
    # specify filtering properties
    filter="IssueCouponRate ge 2",
    # select only the required fields
    select='CommonName, IssuerCommonName, IssueCouponRate, Currency, IssuerCountryName',
    # then specify number of items to be 10000, which is the max; default value is 100
    top=10000
).get_data().data.df

bondsSerarch.head()

,CommonName,IssuerCommonName,IssueCouponRate,Currency,IssuerCountryName
0,TTN 2.375 11/16/24 '24,Titan Global Finance PLC,2.375,EUR,United Kingdom
1,BNP 4.000 10/21/23,Banca Nazionale del Lavoro SpA,4.0,EUR,Italy


In [65]:
equitySerarch = search.Definition(
    view=search.Views.EQUITY_QUOTES,  # for info on `SearchViews`, you can use `help(search.Views)`
    filter="MktCapCompanyUsd ge 1000000000000",
    # select only the required fields
    select='CommonName, PriceCloseUsd, MktCapCompanyUsd',
    # then specify number of items to be 10000, which is the max; default value is 100
    top=100
).get_data().data.df

equitySerarch.head()

,CommonName,PriceCloseUsd,MktCapCompanyUsd
0,APPLE ORD,151.03,2389588504380
1,MICROSOFT ORD,255.29,1900328603939
2,ALPHABET CL A ORD,93.65,1201583710000
3,SAUDI ARABIAN OIL ORD,8.526739,1875882650750
4,ALPHABET CL C ORD,94.02,1201583710000


In [66]:
MnA = search.Definition(
    view=search.Views.DEALS_MERGERS_AND_ACQUISITIONS,  # for info on `SearchViews`, you can use `help(search.Views)`

    # specify filtering properties
    filter="((AcquirerCompanyName ne 'Creditors' and AcquirerCompanyName ne 'Shareholder') and (TargetCountry eq 'US' or TargetCountry eq 'UK')"
    + "and TransactionValueIncludingNetDebtOfTarget ge 100 and TargetPublicStatus eq 'Public')"
    + "and (TransactionStatus eq 'Completed' or TransactionStatus eq 'Pending' or TransactionStatus eq 'Withdrawn')"
    + "and (FormOfTransactionName xeq 'Merger' or FormOfTransactionName xeq 'Acquisition') and (TransactionAnnouncementDate le 2021-11-15 and TransactionAnnouncementDate ge 2020-09-15)",

    # select only the required fields and order them based on announcement date
    # then specify number of items to be 10000, which is the max; default value is 100
    select='TransactionAnnouncementDate, TargetCompanyName, TargetRIC',
    order_by='TransactionAnnouncementDate desc',
    top=10000
).get_data()
MnA = MnA.data.df

#remove companies which doesn't have RIC
MnA = MnA.dropna(subset = ['TargetRIC']).reset_index(drop = True)

print(f'Number of M&A deals for the specified period is {len(MnA)}')
MnA.head()

Number of M&A deals for the specified period is 326


,TransactionAnnouncementDate,TargetCompanyName,TargetRIC
0,2021-11-15,CyrusOne Inc,[CONE.O^C22]
1,2021-11-15,CoreSite Realty Corp,[COR^L21]
2,2021-11-15,LAACO Ltd,[LAACZ.PK^L21]
3,2021-11-15,Casper Sleep Inc,[CSPR.K^A22]
4,2021-11-08,McAfee Corp,[MCFE.O^C22]


A more complex method may be to create a function that incorporates that search function, outputing an excel file. [This](https://gist.github.com/johnukfr/241c2b360a30f96371f430005c8d7738) is a real-life use case replicating RSearch functionality's in looking for Government and Corporate Debt Instruments; the function is rather large, so we put it online for you to access.

The RD Library has also added a number of ease-of-use features as the Search API is very comprehensive - but with that comes some complexity. Search Templates is one such useful feature. 

The "Mines" and "VesselsBoundFor" templates shown here are template examples that you can modify and adapt to your needs. They are defined in the "search-templates.config.json" configuration file. Below is a brief description of the parameters you can use for each template definition.

#### Search templates are defined using the following JSON structure
- **description** (string): Description of the template.
- **parameters** (object): Parameters of the template. These parameters can be used as placeholders in the request_body of the template and as named parameters of the search() method of the template. Each parameter has a name, a description, and an optional default value.
- **request_body** (object): Body of the request sent to the Search REST API. This request body can include template parameters surrounded by '#{' and '}' (for example:  “#{my_param}”). More details on the structure of the request body can be found in the [Search API reference guide](https://apidocs.refinitiv.com/Apps/ApiDocs#/details/L2Rpc2NvdmVyeS9zZWFyY2gvdjE=/Lw==/POST/README) (access to this guide requires a Refinitiv Workspace account or RDP account to log in).
  
##### Template example
  
```json
"MyTemplate": {
    "description": "Search template example.",
    "parameters": {
        "name": {
            "description": "Name of the person to search for.",
            "default": "Edison"
        }
    },
    "request_body": {
       "Query": "#{name}",
       "View":"People",
       "Top":10
    }
} 
```  

**Example of usage**

```  
rd.discovery.search_templates["MyTemplate"].search(name="Pasteur")
```  
  

In [67]:
from physical_assets_map import PhysicalAssetsMap

In [ ]:
#some imports to start with
from physical_assets_map import PhysicalAssetsMap

In [68]:
#load the search templates configuration file
rd.load_config("search-templates.config.json")

<ConfigurationSet: 0x7fe91a3c3908>

##### Mine Search Template example

In [69]:
mines = rd.discovery.search_templates["Mines"]

##### Get help about this template

In [70]:
help(mines)

Help on DiscoverySearchTemplate in module refinitiv.data.discovery._search_templates.search object:

class DiscoverySearchTemplate(refinitiv.data.discovery._search_templates.base.TargetTemplate)
 |  DiscoverySearchTemplate(name=None, *, placeholders_defaults: Union[Dict[str, Any], NoneType] = None, pass_through_defaults: Union[Dict[str, Any], NoneType] = None, optional_placeholders: Iterable[str] = None, ns: 'Namespace' = None, **search_defaults)
 |  
 |  Discovery search preset class
 |  
 |  Method resolution order:
 |      DiscoverySearchTemplate
 |      refinitiv.data.discovery._search_templates.base.TargetTemplate
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  search(self, **kwargs) -> pandas.core.frame.DataFrame
 |      Please, use help() on a template object itself to get method documentation
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from refinitiv.data

##### Use the predefined parameters of the template to search for gold mines in south Africa

In [71]:
from ipyleaflet import Map
mines_coordinates = mines.search(commodity="Gold", region="South Africa")

PhysicalAssetsMap().plot(mines_coordinates).show()
display(mines_coordinates)

Map(center=[-26.416700363159, 27.666700363159], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zo…

,RIC,RCSRegionLeaf,RCSCommodityTypeLeaf,DTSubjectName,PhysicalAssetStatus,Latitude,Longitude
0,C}PX7309413492,South Africa,Gold,South Deep,Normal Operation,-26.4167,27.6667
1,C}PX7309413493,South Africa,Gold,Target,Normal Operation,-27.76,26.629999
2,C}PX7309413494,South Africa,Gold,Tshepong,Normal Operation,-27.866699,26.6833
3,C}PX7309413495,South Africa,Gold,Tau Lekoa,Normal Operation,-27.0,26.613001
4,C}PX7309413496,South Africa,Gold,Kopanang,Normal Operation,-26.98,26.745001
...,...,...,...,...,...,...,...
88,C}PX7310050444,South Africa,Gold,Bloemhoek,Feasibility,-27.534979,27.196499
89,C}PX7310050445,South Africa,Gold,De Bron Merriespruit,Feasibility,-27.571728,27.19503
90,C}PX7310050461,South Africa,Gold,Moab MOD,Normal Operation,-26.989622,26.802376
91,C}PX7310054031,South Africa,Gold,Far West Gold Recoveries (FWGR),Normal Operation,-26.389465,27.360319


## News

### Streaming News

The Office COM API allowed for streaming real time news. This was rather inconvenient because one would have to (i) record news flowing through on their own environment, (ii) be recording with an open streaming session when the news they're looking for is published, (iii) deal with the memory management involved with such streaming data. The Python code below recreated this, but takes a snapshot at the time the code is run:

(N.B: The Accessing News data with the RD or EDAPI Python libraries will only allow you access to news data up to three months ago. For news data predating that, look into [this article](https://developers.refinitiv.com/en/article-catalog/article/news-sentiments---gleaning-insights).)

In [72]:
#Define Callback for stream
def display_data(data, instrument, stream):
    clear_output(wait=True)
    current_time = datetime.datetime.now().time()
    print(current_time, "- Data received for", instrument)
    display(data)

In [73]:
# This will trigger the stream:
stream = rd.open_pricing_stream(
    universe=['NFCP_UBMS'],
    on_data=display_data)

11:21:11.353212 - Data received for NFCP_UBMS


,NEWSCODE18,PNAC,SEQ_NO,TAKE_SEQNO,NEWSCODE20,SYS_DUPID,AREA_ID,ATTRIBTN,PRODCODE_N,PROD_PERM,NEWSCODE13,PROC_DATE,NEWSCODE11,SRCOFDATA,NEWS_PRIO,STORY_TYPE,CROSS_REF,STORY_DATE,NEWSCODE15,RECORDTYPE,DSPLY_NAME,SF_NAME,HEADLINE1,NEWSCODE01,NEWSCODE02,NEWSCODE03,NEWSCODE04,NEWSCODE05,NEWSCODE06,NEWSCODE07,NEWSCODE08,NEWSCODE09,NEWSCODE10,NEWSCODE12,NEWSCODE14,NEWSCODE16,NEWSCODE17,NEWSCODE19,TOT_SEGS,HEAD_DIR,HEAD_LANG,NEWSMGTSTG,DSO_ID,TAKETM_MS,STORYTM_MS,CF_NAME
NFCP_UBMS,None,nBER3DRDqJ,543729,1,None,news_bma,3,BER,BER,457,None,2023-03-06,None,3,3,S,None,2023-03-06,None,232,30,bma,CRUDE PALM OIL FUTURES CLOSING: MARCH 6,A:4 G:1 G:2EK G:8S G:CI G:K G:S M:1QD M:2CM M:...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,L,en,2,52,40871439,40871327,30


In [74]:
stream.close()

<OpenState.Closed: 'Closed'>

#### Selecting Only a Subset of Fields
Above we are dumping all the fields - you may only be interested in a subset of fields - if so you can manage as below:

In [75]:
stream = rd.open_pricing_stream(
    universe=['NFCP_UBMS'],
    fields=['PNAC','HEADLINE1'],
    on_data=display_data)

11:21:21.862759 - Data received for NFCP_UBMS


,PNAC,HEADLINE1
NFCP_UBMS,nNSEsrY7f,APOLLOTYRE-Loss of Share Certificates


In [76]:
stream.close()

<OpenState.Closed: 'Closed'>

#### NEW FUNCTIONALITY: Get News headlines

We added this functionality to find the headlines of news which was not a functionality in the old COM API:

In [77]:
newsResponse = rd.news.get_headlines(query='LSEG',start="20.03.2022",end="25.03.2022",count=3)
newsResponse

,headline,storyId,sourceCode
versionCreated,,,
2022-03-24 18:27:20.000,BRIEF-London Stock Exchange Secondary ABO Book...,urn:newsml:newswire.refinitiv.com:20220324:nFW...,NS:RTRS
2022-03-24 18:09:06.000,BRIEF-London Stock Exchange Secondary ABO Book...,urn:newsml:newswire.refinitiv.com:20220324:nFW...,NS:RTRS
2022-03-24 17:56:21.654,BLOCKTRADE: LONDON STOCK EXCHANGE SECONDARY A...,urn:newsml:newswire.refinitiv.com:20220324:nFW...,NS:RTRS


### Historical News

Thankfully, the `ek` library News API is much more powerful and simple to use. Again, most of it is better shown and explained on [GitHub](https://github.com/Refinitiv-API-Samples/Example.DataLibrary.Python/tree/main/Examples), but the below is a great starting point of how to collect News data (that is 15 months old or older):

In [78]:
story_id = newsResponse.iat[1,1]
story_id

'urn:newsml:newswire.refinitiv.com:20220324:nFWN2VR17K:1'

In [79]:
from IPython.display import HTML
story = rd.news.get_story(story_id)
HTML(story)

## Contribution - AdxRtContribute.dll

**CONTRIBUTIONS WORKFLOW IN PYTHON IS UPCOMING.** Keep an eye on this article, it will be updates with such Python use-cases.

### What does AdxRtContribute.dll do?

#### VBA

AdxRtContribute is for data contribution either locally (SCOPE:LOCAL) within the instance of Excel (or application if EikonDesktopDataAPI has been used) or on to a server (SCOPE:SERVER) and potentially on to IDN (where the Source Alias of "TRC" should be specified).

AdxRtContribute replicates the RtContribute() function in Eikon for Excel or Eikon - Microsoft Office.

NOTE - Eikon for Excel or Eikon - Microsoft Office should be logged in otherwise the sample VBA code will generate an error "ERROR #360c - AdxRtx : No connection to the platform".

**Prerequisites**

    1. Open a new single sheet Excel workbook.
       Save As with an appropriate name (e.g. AdxRtContribute.xls or AdxRtContribute.xlsm in Office 2007 or higher).
       
    2. Go to the VBE (Visual Basic Editor), ensure the Project Explorer is visible and select the project for the workbook above.
       <ALT><F11> or Tools, Macro, Visual Basic Editor in Excel 2003 or  Developer, Visual Basic in Excel 2007 and above, View, Project Explorer If the Developer header is not visible in Excel 2007 and above, go to the Excel Office Button, select Excel Options (lower right), Popular, and check the 'Show Developer tab in the Ribbon' box.
    3. In the VBE, click on File, Import File and import PLVbaApis.bas.
       The .bas location is C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin (Z may be X or Y, depending on the last Eikon update). The .bas is loaded as a new VB project module, PLVbaApis.
    4. In the PLVbaAPis module, comment out the sections which aren't required.
       As this example is dealing with AdxRtContribute, part of the real time library AdfinXRtLib, the AdfinX Real Time section can remain uncommented.
    5. In the VBE, go Tools, References and ensure that AdfinX Real Time Library is checked.
       If it is not in the list the library is called rtx.dll and its location for Eikon 4 is >C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin (Z may be X or Y, depending on the last Eikon update).
    AdxRtContribute

image?

    6. Create an instance of an AdxRtContribute object using the PLVbaApis function CreateAdxRtContribute.
> Set myAdxRtContrib = CreateAdxRtContribute

    7. To replicate RtContribute(), when the object has been created set the source, item name and mode (if required) and then contribute the field(s) and value(s) desired.
    
>     Public myAdxRtContrib As AdfinXRtLib.AdxRtContribute
>
>     Dim fields (1 To 3) As Variant
>     Dim values (1 To 3) As Variant
>
>     Sub myContribution()
>       Set myAdxRtContrib = CreateAdxRtContribute
>       If myAdxRtContrib Is Nothing Then MsgBox "Failed to create RtContribute"
>         With myAdxRtContrib
>           .Source = "DAVID2"
>           .ItemName = "TEST_RIC2"
>           .Mode = "SCOPE:LOCAL UPDATE:CHANGED"
>
>           fields (1)="BID"
>           fields (2)="ASK"
>           fields (3)="ASKROW80_1"
>           values(1)=210
>           values(2)=220
>           values(3)="ABCDEF"
>
>           .Contribute fields, values
>           '.Contribute "ASK", 220 ' Alternative method of actual names and values'
>           '.Contribute Array("BID", "ASK"), Array(210, 220) ' Another alternative method'
>        End With
>     End Sub

    8. RtGet() or RData() can be used in Eikon for Excel to check that the contribution has been successful. If the AdxRtList COM API for real time data retrieval is being used this can be set with the appropriate source and instrument parameters.

    9. If no more contributions are required the AdxRtContribute object can be closed (set to Nothing).
    
#### Python

**CONTRIBUTIONS WORKFLOW IN PYTHON IS UPCOMING.** Keep an eye on this article, it will be updates with such Python use-cases.

## Conclusion

In conclusion, we can see that the [Office COM API](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office) had many great uses, but limitations too. This was without mentioning its reliability on [DLL](https://docs.microsoft.com/en-us/troubleshoot/windows-client/deployment/dynamic-link-library)s that can be heavy to run on a personal machine. But the Refinitiv Python Libraries ([RD](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python), [RDP](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis) and [EDAPI](https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api)) can not only replicate these COM functionalities but enhance them in many instances, the simplest example being the Historical News functionality shown above.

Several COM API functionalities relying on a technology called Adfin was not replicated in Python in this article, but we will investigate them in another article - so stay tuned!

# Further Information

## COM Prerequisites


1. Open a new single sheet Excel workbook.

    Save As with an appropriate name (e.g. AdxRtSourceList.xls or AdxRtSourceList.xlsm in Office 2007 or higher).
    
2. Go to the VBE (Visual Basic Editor), ensure the Project Explorer is visible and select the project for the workbook.

    \<ALT\>\<F11\> or Tools, Macro, Visual Basic Editor in Excel 2003 or  Developer, Visual Basic in Excel 2007 and above, View, Project Explorer If the Developer header is not visible in Excel 2007 and above, go to the Excel Office Button, select Excel Options (lower right), Popular, and check the 'Show Developer tab in the Ribbon' box.
    
3. In the VBE, click on File, Import File and import PLVbaApis.bas.
    
    The .bas location is C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin (Z may be X or Y, depending on the last Eikon update). The .bas is loaded as a new VB project module, PLVbaApis.
    
4. In the PLVbaAPis module, comment out the sections which aren't required.
    
    E.G.: when dealing with AdxRtSourceList, part of the real time library AdfinXRtLib, the AdfinX Real Time section can remain uncommented.

5. In the VBE, go Tools, References and ensure that AdfinX Real Time Library is checked.
    
    If it is not in the list the library is called rtx.dll and its location for Eikon 4 is ">C:\Program Files (x86)\Thomson Reuters\Eikon\Z\Bin (Z may be X or Y, depending on the last Eikon update).
    
## Further Resources

COM APIs:
[Overview](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office) | [Quickstart Guide](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/quick-start) | [Documentation](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/documentation) | [Downloads](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/download) | [Tutorials](https://developers.refinitiv.com/en/api-catalog/eikon/com-apis-for-use-in-microsoft-office/tutorials) | [Q&A Forum](https://community.developers.refinitiv.com/index.html?_ga=2.221580381.456015790.1678102215-1923062744.1661858778)

RD Library: 
[Overview](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) | [Quickstart Guide](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start) | [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation) | [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials) | [Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html?_ga=2.149760115.456015790.1678102215-1923062744.1661858778)